# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol
import psycopg2
import scipy.interpolate as intp

In [2]:
import os

In [3]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold

In [4]:
df3_fold = 'D:/data/big2/BD3/df'

# server

In [5]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [6]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')

# Load

## 등록정보(STD_CEG_CAR_MIG)

In [7]:
# exasol db
# 4m 21s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG;")
# test 만개
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE ROWNUM <= 10000;")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [8]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [9]:
carr = car.rename(columns=car_ch_col)

In [10]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [11]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [12]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [13]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [14]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_NM  47916 non-null  object
 2   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [15]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [16]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [17]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [18]:
# 1m 13.2s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

In [19]:
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO FROM from vsysd.ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in cur.description]
# cur.close()
# conn.close()

# 전처리

## 중복 차대번호 제거

In [20]:
carr.shape, len(carr['차대번호'].unique())

((29998033, 11), 29984639)

In [21]:
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(29984639, 11)

## 배출가스등급 코드 변환

In [22]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['X', '2', '1', '3', '5', '4', nan], dtype=object)

In [23]:
carr['배출가스등급'].value_counts(dropna=False)

배출가스등급
2      12174611
3      10050952
1       2940294
5       2377906
4       1897506
X        543339
NaN          31
Name: count, dtype: int64

## 차종 코드 변환

In [24]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

## 용도 코드 변환

In [25]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [26]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(26203796, 11)

## 연료 코드 변환

In [27]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '알코올', '전기', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [28]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [29]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(26203796, 17)

## 등록&제원&법정동코드 병합(df)

In [30]:
df = cs.merge(coder, on='법정동코드', how='left')

In [31]:
df.shape

(26203796, 19)

### 매칭 안되는 지역 처리

In [32]:
df['시도'].isnull().sum()

2

In [33]:
df.loc[df['시도'].isnull() == True, ['법정동코드', '시도', '시군구']]

,법정동코드,시도,시군구
25189713,4165052000,NaN,NaN
25946148,4163055000,NaN,NaN


In [34]:
df.loc[df['시도'].isnull() == True, '법정동코드'].unique()

array([4165052000, 4163055000], dtype=int64)

In [35]:
# 주소 수정
# df.loc[df['법정동코드'] == 4161011600, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 능평동
# df.loc[df['법정동코드'] == 4161012000, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 양벌동
# df.loc[df['법정동코드'] == 4161011800, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 추자동
# df.loc[df['법정동코드'] == 4161011500, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 신현동
# df.loc[df['법정동코드'] == 4161011900, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 매산동
# df.loc[df['법정동코드'] == 4161011700, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 문형동
# df.loc[df['법정동코드'] == 4161011400, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 고산동
df.loc[df['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

In [36]:
df['시도'].isnull().sum()

0

## 등록(말소 유지) & 제원 병합

In [37]:
# 10.3s
cse = carr.merge(srcr, on='제원관리번호', how='left')

In [38]:
cse.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'],
      dtype='object')

## 등록&제원&이력 병합

In [39]:
# 2m 6.0s
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [40]:
ersr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자'],
      dtype='object')

In [41]:
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [42]:
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [43]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

## 등록&제원&이력&법정동코드 병합(errc)

In [44]:
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

# 분석

## 연료 하이브리드 수정

In [45]:
# about 23.6s
df.loc[(df['연료'] == '휘발유') | (df['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
df.loc[(df['연료'] == '경유') | (df['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
df.loc[(df['연료'] == 'LPG(액화석유가스)') | (df['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
df.loc[(df['연료'] == 'CNG(압축천연가스)') | (df['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
df.loc[(df['연료'] == 'LNG(액화천연가스)') | (df['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
df.loc[df['연료'] == '전기', 'fuel'] = '전기'
df.loc[df['연료'] == '수소', 'fuel'] = '수소'
df.loc[df['연료'] == '태양열', 'fuel'] = '태양열'
df.loc[df['연료'] == '알코올', 'fuel'] = '알코올'
df.loc[df['연료'] == '등유', 'fuel'] = '등유'
df.loc[df['연료'] == '기타연료', 'fuel'] = '기타연료'
df.loc[df['연료'].isnull() == True, 'fuel'] = np.nan

In [46]:
# about 3.3s
errc.loc[(errc['연료'] == '휘발유') | (errc['연료'] == '휘발유 하이브리드'), 'fuel'] = '휘발유'
errc.loc[(errc['연료'] == '경유') | (errc['연료'] == '경유 하이브리드'), 'fuel'] = '경유'
errc.loc[(errc['연료'] == 'LPG(액화석유가스)') | (errc['연료'] == 'LPG 하이브리드'), 'fuel'] = 'LPG'
errc.loc[(errc['연료'] == 'CNG(압축천연가스)') | (errc['연료'] == 'CNG 하이브리드'), 'fuel'] = 'CNG'
errc.loc[(errc['연료'] == 'LNG(액화천연가스)') | (errc['연료'] == 'LNG 하이브리드'), 'fuel'] = 'LNG'
errc.loc[errc['연료'] == '전기', 'fuel'] = '전기'
errc.loc[errc['연료'] == '수소', 'fuel'] = '수소'
errc.loc[errc['연료'] == '태양열', 'fuel'] = '태양열'
errc.loc[errc['연료'] == '알코올', 'fuel'] = '알코올'
errc.loc[errc['연료'] == '등유', 'fuel'] = '등유'
errc.loc[errc['연료'] == '기타연료', 'fuel'] = '기타연료'
errc.loc[errc['연료'].isnull() == True, 'fuel'] = np.nan

In [47]:
# 6s
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
df.loc[(df['시도'] == '서울특별시') | (df['시도'] == '인천광역시') | (df['시도'] == '경기도'), '지역'] = '수도권'
df['지역'] = df['지역'].fillna('비수도권')

In [48]:
dfm = df.copy()

In [49]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [50]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

In [51]:
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
errc.loc[(errc['시도'] == '서울특별시') | (errc['시도'] == '인천광역시') | (errc['시도'] == '경기도'), '지역'] = '수도권'
errc['지역'] = errc['지역'].fillna('비수도권')

In [52]:
errc['변경일자'].isnull().sum()

0

In [53]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]
errc['변경일자'] = errc['변경일자_년'] + errc['변경일자_월'] + errc['변경일자_일']

In [54]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [55]:
# 2m 41.5s
dfm2 = dfm.loc[(dfm['fuel'] == '경유') | (dfm['fuel'] == '휘발유') | (dfm['fuel'] == 'LPG') | (dfm['fuel'] == '전기') | (dfm['fuel'] == '수소')].reset_index(drop=True)
errc2 = errc.loc[(errc['fuel'] == '경유') | (errc['fuel'] == '휘발유') | (errc['fuel'] == 'LPG') | (errc['fuel'] == '전기') | (errc['fuel'] == '수소')].reset_index(drop=True)

## 등급, 지역별 차량현황

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230721'

In [ ]:
# 데이터 연도 설정
year = 2022
# year = int(today_date[:4])

In [ ]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '지역', '시도', '배출가스등급', '차량대수']]
grp1

,연도,지역,시도,배출가스등급,차량대수
0,2022,비수도권,강원도,1,80078
1,2022,비수도권,강원도,2,358656
2,2022,비수도권,강원도,3,310061
3,2022,비수도권,강원도,4,54669
4,2022,비수도권,강원도,5,47146
...,...,...,...,...,...
97,2022,수도권,인천광역시,2,793987
98,2022,수도권,인천광역시,3,580695
99,2022,수도권,인천광역시,4,77356
100,2022,수도권,인천광역시,5,52907


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
rgn_list = []
ctpv_list = []
grd_list = []
for ctpv in grp1['시도'].unique():
    if ('서울특별시' in ctpv) or ('인천광역시' in ctpv) or ('경기도' in ctpv):
        rgn = '수도권'
    else:
        rgn = '비수도권'
        for grd in ['1', '2', '3', '4', '5', 'X']:
            for yr in range(2019, year + 1):
                yr_list.append(str(yr))
                rgn_list.append(rgn)
                ctpv_list.append(ctpv)
                grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '지역':rgn_list, '시도':ctpv_list, '배출가스등급':grd_list})

In [ ]:
base.head()

,연도,지역,시도,배출가스등급
0,2019,비수도권,강원도,1
1,2020,비수도권,강원도,1
2,2021,비수도권,강원도,1
3,2022,비수도권,강원도,1
4,2019,비수도권,강원도,2


In [ ]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,지역,시도,배출가스등급,등록대수
0,1955,비수도권,강원도,5,1
1,1958,비수도권,경상북도,5,1
2,1962,비수도권,강원도,5,4
3,1962,비수도권,경상북도,5,1
4,1962,비수도권,전라북도,5,1
...,...,...,...,...,...
3066,2023,수도권,서울특별시,X,29242
3067,2023,수도권,인천광역시,1,8305
3068,2023,수도권,인천광역시,2,14541
3069,2023,수도권,인천광역시,3,4722


In [ ]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,지역,시도,배출가스등급,말소대수
0,2019,비수도권,강원도,1,220
1,2019,비수도권,강원도,2,4208
2,2019,비수도권,강원도,3,8191
3,2019,비수도권,강원도,4,2557
4,2019,비수도권,강원도,5,11786
...,...,...,...,...,...
401,2022,수도권,인천광역시,2,9879
402,2022,수도권,인천광역시,3,10842
403,2022,수도권,인천광역시,4,3113
404,2022,수도권,인천광역시,5,4507


In [ ]:
base1 = base.merge(grp1, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base1

,연도,지역,시도,배출가스등급,차량대수
0,2019,비수도권,강원도,1,NaN
1,2020,비수도권,강원도,1,NaN
2,2021,비수도권,강원도,1,NaN
3,2022,비수도권,강원도,1,80078.00
4,2019,비수도권,강원도,2,NaN
...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40821.00
332,2019,비수도권,충청북도,X,NaN
333,2020,비수도권,충청북도,X,NaN
334,2021,비수도권,충청북도,X,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base2

,연도,지역,시도,배출가스등급,차량대수,등록대수
0,2019,비수도권,강원도,1,NaN,7882.00
1,2020,비수도권,강원도,1,NaN,11940.00
2,2021,비수도권,강원도,1,NaN,14037.00
3,2022,비수도권,강원도,1,80078.00,10891.00
4,2019,비수도권,강원도,2,NaN,23234.00
...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40821.00,1.00
332,2019,비수도권,충청북도,X,NaN,27.00
333,2020,비수도권,충청북도,X,NaN,38.00
334,2021,비수도권,충청북도,X,NaN,29.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,NaN,7882.00,220.00
1,2020,비수도권,강원도,1,NaN,11940.00,461.00
2,2021,비수도권,강원도,1,NaN,14037.00,854.00
3,2022,비수도권,강원도,1,80078.00,10891.00,501.00
4,2019,비수도권,강원도,2,NaN,23234.00,4208.00
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40821.00,1.00,4899.00
332,2019,비수도권,충청북도,X,NaN,27.00,37.00
333,2020,비수도권,충청북도,X,NaN,38.00,26.00
334,2021,비수도권,충청북도,X,NaN,29.00,13.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,45026.00,7882.00,220.00
1,2020,비수도권,강원도,1,56505.00,11940.00,461.00
2,2021,비수도권,강원도,1,69688.00,14037.00,854.00
3,2022,비수도권,강원도,1,80078.00,10891.00,501.00
4,2019,비수도권,강원도,2,315866.00,23234.00,4208.00
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,40821.00,1.00,4899.00
332,2019,비수도권,충청북도,X,8344.00,27.00,37.00
333,2020,비수도권,충청북도,X,8356.00,38.00,26.00
334,2021,비수도권,충청북도,X,8372.00,29.00,13.00


In [ ]:
base3.columns

Index(['연도', '지역', '시도', '배출가스등급', '차량대수', '등록대수', '말소대수'], dtype='object')

In [ ]:
df1 = base3[['연도', '지역', '시도', '배출가스등급', '차량대수']]

In [59]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230721'

In [ ]:
df1['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8372\2264777512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['테이블생성일자'] = today_date


In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '지역':'RGN', 
    '시도':'CTPV', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [ ]:
STD_BD_CAR_CURSTT_MOD = df1.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD.columns

Index(['YR', 'RGN', 'CTPV', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT', 'LOAD_DT'], dtype='object')

In [ ]:
STD_BD_CAR_CURSTT_MOD.head()

,YR,RGN,CTPV,EXHST_GAS_GRD_CD,VHCL_MKCNT,LOAD_DT
0,2019,비수도권,강원도,1,45026.00,20230721
1,2020,비수도권,강원도,1,56505.00,20230721
2,2021,비수도권,강원도,1,69688.00,20230721
3,2022,비수도권,강원도,1,80078.00,20230721
4,2019,비수도권,강원도,2,315866.00,20230721


### [출력] STD_BD_CAR_CURSTT_MOD

In [ ]:
# expdf = STD_BD_CAR_CURSTT_MOD
# table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 7.6s
# STD_BD_CAR_CURSTT_MOD.to_csv(os.path.join(df3_fold, 'STD_BD_CAR_CURSTT_MOD.csv'), index=False)

## 등급, 연료별 차량현황

In [ ]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel', '배출가스등급', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,배출가스등급,차량대수
0,2022,LPG,1,479005
1,2022,LPG,2,1176931
2,2022,LPG,3,238483
3,2022,LPG,4,24592
4,2022,LPG,5,436


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel'].unique():
    for grd in ['1', '2', '3', '4', '5', 'X']:
        for yr in range(2019, year + 1):
            yr_list.append(str(yr))
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [ ]:
base.head()

,연도,fuel,배출가스등급
0,2019,LPG,1
1,2020,LPG,1
2,2021,LPG,1
3,2022,LPG,1
4,2019,LPG,2


In [ ]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,배출가스등급,등록대수
0,1955,경유,5,1
1,1958,휘발유,5,1
2,1962,경유,5,8
3,1962,휘발유,5,1
4,1963,경유,5,2
...,...,...,...,...
478,2023,전기,1,34858
479,2023,전기,X,15135
480,2023,휘발유,1,72399
481,2023,휘발유,2,157626


In [ ]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,배출가스등급,말소대수
0,2019,LPG,1,5974
1,2019,LPG,2,27864
2,2019,LPG,3,42824
3,2019,LPG,4,3460
4,2019,LPG,5,3
...,...,...,...,...
71,2022,휘발유,2,116723
72,2022,휘발유,3,94401
73,2022,휘발유,4,11550
74,2022,휘발유,5,206


In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel', '배출가스등급'], how='left')
base1

,연도,fuel,배출가스등급,차량대수
0,2019,LPG,1,NaN
1,2020,LPG,1,NaN
2,2021,LPG,1,NaN
3,2022,LPG,1,479005.00
4,2019,LPG,2,NaN
...,...,...,...,...
115,2022,휘발유,5,30011.00
116,2019,휘발유,X,NaN
117,2020,휘발유,X,NaN
118,2021,휘발유,X,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '배출가스등급'], how='left')
base2

,연도,fuel,배출가스등급,차량대수,등록대수
0,2019,LPG,1,NaN,62979.00
1,2020,LPG,1,NaN,47598.00
2,2021,LPG,1,NaN,36005.00
3,2022,LPG,1,479005.00,16628.00
4,2019,LPG,2,NaN,65541.00
...,...,...,...,...,...
115,2022,휘발유,5,30011.00,NaN
116,2019,휘발유,X,NaN,2659.00
117,2020,휘발유,X,NaN,4612.00
118,2021,휘발유,X,NaN,5592.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '배출가스등급'], how='left')
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,NaN,62979.00,5974.00
1,2020,LPG,1,NaN,47598.00,15886.00
2,2021,LPG,1,NaN,36005.00,23771.00
3,2022,LPG,1,479005.00,16628.00,10239.00
4,2019,LPG,2,NaN,65541.00,27864.00
...,...,...,...,...,...,...
115,2022,휘발유,5,30011.00,NaN,206.00
116,2019,휘발유,X,NaN,2659.00,305.00
117,2020,휘발유,X,NaN,4612.00,773.00
118,2021,휘발유,X,NaN,5592.00,1931.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,428670.00,62979.00,5974.00
1,2020,LPG,1,460382.00,47598.00,15886.00
2,2021,LPG,1,472616.00,36005.00,23771.00
3,2022,LPG,1,479005.00,16628.00,10239.00
4,2019,LPG,2,1102311.00,65541.00,27864.00
...,...,...,...,...,...,...
115,2022,휘발유,5,30011.00,0.00,206.00
116,2019,휘발유,X,174657.00,2659.00,305.00
117,2020,휘발유,X,178496.00,4612.00,773.00
118,2021,휘발유,X,182157.00,5592.00,1931.00


In [ ]:
df2 = base3[['연도', 'fuel', '배출가스등급', '차량대수']]

In [ ]:
df2['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8372\2409603865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['테이블생성일자'] = today_date


In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [ ]:
STD_BD_CAR_CURSTT_MOD2 = df2.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD2.columns

Index(['YR', 'fuel', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT', 'LOAD_DT'], dtype='object')

In [ ]:
STD_BD_CAR_CURSTT_MOD2.head()

,YR,fuel,EXHST_GAS_GRD_CD,VHCL_MKCNT,LOAD_DT
0,2019,LPG,1,428670.00,20230721
1,2020,LPG,1,460382.00,20230721
2,2021,LPG,1,472616.00,20230721
3,2022,LPG,1,479005.00,20230721
4,2019,LPG,2,1102311.00,20230721


### [출력] STD_BD_CAR_CURSTT_MOD2

In [ ]:
# expdf = STD_BD_CAR_CURSTT_MOD2
# table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 7.6s
# STD_BD_CAR_CURSTT_MOD2.to_csv(os.path.join(df3_fold, 'STD_BD_CAR_CURSTT_MOD2.csv'), index=False)

## 전체 차량 현황(등급, 차종, 지역별 차량현황)

In [ ]:
dfm2['테이블생성일자'] = today_date

In [ ]:
df3 = dfm2[[
    '테이블생성일자', 
    '법정동코드', 
    '차종',
    '용도', 
    '차대번호', 
    '제원관리번호', 
    '배출가스인증번호', 
    '배출가스등급', 
    '차량말소YN', 
    'fuel', 
    '시도', 
    '시군구_수정'
]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT',
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD',
    '용도':'PURPS_CD2', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량말소YN':'VHCL_ERSR_YN', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
}

In [ ]:
STD_BD_CAR_CURSTT = df3.rename(columns=cdict)
STD_BD_CAR_CURSTT.columns

Index(['LOAD_DT', 'STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'VIN', 'MANG_MNG_NO',
       'EXHST_GAS_CERT_NO', 'EXHST_GAS_GRD_CD', 'VHCL_ERSR_YN', 'FUEL_CD',
       'CTPV', 'SGG'],
      dtype='object')

### [출력] STD_BD_CAR_CURSTT

In [ ]:
# expdf = STD_BD_CAR_CURSTT
# table_nm = 'STD_BD_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1m 56.2s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 1m 47.5s
# STD_BD_CAR_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_CAR_CURSTT.csv'), index=False)

## 지역, 연료, 연도별 차량 현황 분석

In [ ]:
# 2022년 차량 대수
grp1 = dfm2.groupby(['fuel', '시도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel', '시도', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,시도,차량대수
0,2022,LPG,강원도,59045
1,2022,LPG,경기도,401786
2,2022,LPG,경상남도,142122
3,2022,LPG,경상북도,129474
4,2022,LPG,광주광역시,90117


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
ctpv_list = []
for fuel in grp1['fuel'].unique():
    for ctpv in grp1['시도'].unique():
        for yr in range(2019, year + 1):
            yr_list.append(str(yr))
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '시도':ctpv_list})

In [ ]:
base.head()

,연도,fuel,시도
0,2019,LPG,강원도
1,2020,LPG,강원도
2,2021,LPG,강원도
3,2022,LPG,강원도
4,2019,LPG,경기도


In [ ]:
# 연도별 등록대수
grp2 = dfm2.groupby(['최초등록일자_년', 'fuel', '시도'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,시도,등록대수
0,1955,경유,강원도,1
1,1958,휘발유,경상북도,1
2,1962,경유,강원도,4
3,1962,경유,경기도,2
4,1962,경유,전라북도,1
...,...,...,...,...
2768,2023,휘발유,전라남도,22196
2769,2023,휘발유,전라북도,11390
2770,2023,휘발유,제주특별자치도,19860
2771,2023,휘발유,충청남도,16292


In [ ]:
# 연도별 말소대수
grp3 = errc2.groupby(['변경일자_년', 'fuel', '시도'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,시도,말소대수
0,2019,LPG,강원도,2719
1,2019,LPG,경기도,17973
2,2019,LPG,경상남도,5673
3,2019,LPG,경상북도,4960
4,2019,LPG,광주광역시,3067
...,...,...,...,...
303,2022,휘발유,전라남도,8254
304,2022,휘발유,전라북도,8175
305,2022,휘발유,제주특별자치도,2990
306,2022,휘발유,충청남도,11174


In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel', '시도'], how='left')
base1

,연도,fuel,시도,차량대수
0,2019,LPG,강원도,NaN
1,2020,LPG,강원도,NaN
2,2021,LPG,강원도,NaN
3,2022,LPG,강원도,59045.00
4,2019,LPG,경기도,NaN
...,...,...,...,...
335,2022,휘발유,충청남도,587714.00
336,2019,휘발유,충청북도,NaN
337,2020,휘발유,충청북도,NaN
338,2021,휘발유,충청북도,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '시도'], how='left')
base2

,연도,fuel,시도,차량대수,등록대수
0,2019,LPG,강원도,NaN,3386.00
1,2020,LPG,강원도,NaN,3433.00
2,2021,LPG,강원도,NaN,3003.00
3,2022,LPG,강원도,59045.00,1951.00
4,2019,LPG,경기도,NaN,24300.00
...,...,...,...,...,...
335,2022,휘발유,충청남도,587714.00,31172.00
336,2019,휘발유,충청북도,NaN,29768.00
337,2020,휘발유,충청북도,NaN,35756.00
338,2021,휘발유,충청북도,NaN,31143.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '시도'], how='left')
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원도,NaN,3386.00,2719.00
1,2020,LPG,강원도,NaN,3433.00,4651.00
2,2021,LPG,강원도,NaN,3003.00,4552.00
3,2022,LPG,강원도,59045.00,1951.00,1883.00
4,2019,LPG,경기도,NaN,24300.00,17973.00
...,...,...,...,...,...,...
335,2022,휘발유,충청남도,587714.00,31172.00,11174.00
336,2019,휘발유,충청북도,NaN,29768.00,9894.00
337,2020,휘발유,충청북도,NaN,35756.00,18006.00
338,2021,휘발유,충청북도,NaN,31143.00,18136.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    255
등록대수      3
말소대수     32
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,시도,차량대수,등록대수,말소대수
0,2019,LPG,강원도,61744.00,3386.00,2719.00
1,2020,LPG,강원도,60526.00,3433.00,4651.00
2,2021,LPG,강원도,58977.00,3003.00,4552.00
3,2022,LPG,강원도,59045.00,1951.00,1883.00
4,2019,LPG,경기도,423704.00,24300.00,17973.00
...,...,...,...,...,...,...
335,2022,휘발유,충청남도,587714.00,31172.00,11174.00
336,2019,휘발유,충청북도,404570.00,29768.00,9894.00
337,2020,휘발유,충청북도,422320.00,35756.00,18006.00
338,2021,휘발유,충청북도,435327.00,31143.00,18136.00


In [ ]:
gp = base3.groupby(['연도', '시도', 'fuel'], as_index=False)['차량대수'].sum()

In [ ]:
gp['연료비율'] = round((gp['차량대수'] / gp.groupby(['연도', '시도'])['차량대수'].transform('sum')), 2)

In [ ]:
gp['테이블생성일자'] = today_date

In [ ]:
gp1 = gp[['테이블생성일자', '연도', 'fuel', '시도', '차량대수', '연료비율']]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
    '연료비율':'FUEL_RT'
}

In [ ]:
STD_BD_CAR_REG_MKCNT = gp1.rename(columns=cdict)
STD_BD_CAR_REG_MKCNT.columns

Index(['LOAD_DT', 'YR', 'FUEL_CD', 'CTPV', 'VHCL_MKCNT', 'FUEL_RT'], dtype='object')

In [ ]:
STD_BD_CAR_REG_MKCNT.head()

,LOAD_DT,YR,FUEL_CD,CTPV,VHCL_MKCNT,FUEL_RT
0,20230721,2019,LPG,강원도,61744.00,0.08
1,20230721,2019,경유,강원도,386588.00,0.47
2,20230721,2019,수소,강원도,262.00,0.00
3,20230721,2019,전기,강원도,3833.00,0.00
4,20230721,2019,휘발유,강원도,369908.00,0.45


### [출력] STD_BD_CAR_REG_MKCNT

In [ ]:
# expdf = STD_BD_CAR_REG_MKCNT
# table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 7.6s
# STD_BD_CAR_REG_MKCNT.to_csv(os.path.join(df3_fold, 'STD_BD_CAR_REG_MKCNT.csv'), index=False)

## 내연차 연료, 연도별 차량 현황 예측
- 경유, 휘발유, LPG

In [ ]:
dfm2['연료'].unique()

array(['경유', '휘발유', '전기', '휘발유 하이브리드', 'LPG(액화석유가스)', '경유 하이브리드', '수소',
       'LPG 하이브리드'], dtype=object)

In [ ]:
# 43.5s
dfm2dgl = dfm2.loc[(dfm2['연료'] == '경유') | (dfm2['연료'] == '휘발유') | (dfm2['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
dfm2dgl.shape

(24226476, 25)

In [ ]:
# 6.4s
errc2dgl = errc2.loc[(errc2['연료'] == '경유') | (errc2['연료'] == '휘발유') | (errc2['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
errc2dgl.shape

(3581512, 27)

In [ ]:
# 2022년 차량 대수
grp1 = dfm2dgl.groupby('연료', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '연료', '차량대수']]

In [ ]:
grp1.head()

,연도,연료,차량대수
0,2022,LPG(액화석유가스),1921725
1,2022,경유,9940705
2,2022,휘발유,12364046


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['연료'].unique():
    for yr in range(2019, year + 1):
        yr_list.append(str(yr))
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list})

In [ ]:
base.head()

,연도,연료
0,2019,LPG(액화석유가스)
1,2020,LPG(액화석유가스)
2,2021,LPG(액화석유가스)
3,2022,LPG(액화석유가스)
4,2019,경유


In [ ]:
# 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', '연료'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,연료,등록대수
0,1955,경유,1
1,1958,휘발유,1
2,1962,경유,8
3,1962,휘발유,1
4,1963,경유,2
...,...,...,...
164,2022,경유,269902
165,2022,휘발유,656027
166,2023,LPG(액화석유가스),20773
167,2023,경유,104991


In [ ]:
# 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', '연료'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,연료,말소대수
0,2019,LPG(액화석유가스),79957
1,2019,경유,338073
2,2019,휘발유,280669
3,2020,LPG(액화석유가스),142740
4,2020,경유,557370
5,2020,휘발유,507537
6,2021,LPG(액화석유가스),133782
7,2021,경유,539510
8,2021,휘발유,503646
9,2022,LPG(액화석유가스),55449


In [ ]:
base1 = base.merge(grp1, on=['연도', '연료'], how='left')
base1

,연도,연료,차량대수
0,2019,LPG(액화석유가스),NaN
1,2020,LPG(액화석유가스),NaN
2,2021,LPG(액화석유가스),NaN
3,2022,LPG(액화석유가스),1921725.00
4,2019,경유,NaN
5,2020,경유,NaN
6,2021,경유,NaN
7,2022,경유,9940705.00
8,2019,휘발유,NaN
9,2020,휘발유,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '연료'], how='left')
base2

,연도,연료,차량대수,등록대수
0,2019,LPG(액화석유가스),NaN,128532
1,2020,LPG(액화석유가스),NaN,124962
2,2021,LPG(액화석유가스),NaN,104596
3,2022,LPG(액화석유가스),1921725.00,68244
4,2019,경유,NaN,644913
5,2020,경유,NaN,582623
6,2021,경유,NaN,415503
7,2022,경유,9940705.00,269902
8,2019,휘발유,NaN,843609
9,2020,휘발유,NaN,950441


In [ ]:
base3 = base2.merge(grp3, on=['연도', '연료'], how='left')
base3

,연도,연료,차량대수,등록대수,말소대수
0,2019,LPG(액화석유가스),NaN,128532,79957
1,2020,LPG(액화석유가스),NaN,124962,142740
2,2021,LPG(액화석유가스),NaN,104596,133782
3,2022,LPG(액화석유가스),1921725.00,68244,55449
4,2019,경유,NaN,644913,338073
5,2020,경유,NaN,582623,557370
6,2021,경유,NaN,415503,539510
7,2022,경유,9940705.00,269902,216396
8,2019,휘발유,NaN,843609,280669
9,2020,휘발유,NaN,950441,507537


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    9
등록대수    0
말소대수    0
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,연료,차량대수,등록대수,말소대수
0,2019,LPG(액화석유가스),1955894.00,128532,79957
1,2020,LPG(액화석유가스),1938116.00,124962,142740
2,2021,LPG(액화석유가스),1908930.00,104596,133782
3,2022,LPG(액화석유가스),1921725.00,68244,55449
4,2019,경유,9985953.00,644913,338073
5,2020,경유,10011206.00,582623,557370
6,2021,경유,9887199.00,415503,539510
7,2022,경유,9940705.00,269902,216396
8,2019,휘발유,11143801.00,843609,280669
9,2020,휘발유,11586705.00,950441,507537


In [ ]:
die = base3.loc[base3['연료'] == '경유', ['연도', '연료', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['연료'] == '휘발유', ['연도', '연료', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['연료'] == 'LPG(액화석유가스)', ['연도', '연료', '차량대수']].reset_index(drop=True)

In [ ]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
# 선형 예측
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [ ]:
# BSpline 예측
spl1 = intp.BSpline(die['연도'], die['차량대수'], 1, extrapolate=True)
spl2 = intp.BSpline(gas['연도'], gas['차량대수'], 1, extrapolate=True)
spl3 = intp.BSpline(lpg['연도'], lpg['차량대수'], 1, extrapolate=True)
spl1pred = spl1(range(year + 1, 2036))
spl2pred = spl2(range(year + 1, 2036))
spl3pred = spl3(range(year + 1, 2036))

In [ ]:
# akima 예측
aki1 = intp.Akima1DInterpolator(die['연도'], die['차량대수'])
aki2 = intp.Akima1DInterpolator(gas['연도'], gas['차량대수'])
aki3 = intp.Akima1DInterpolator(lpg['연도'], lpg['차량대수'])
aki1pred = aki1([x for x in range(year + 1, 2036)], extrapolate=True)
aki2pred = aki2([x for x in range(year + 1, 2036)], extrapolate=True)
aki3pred = aki3([x for x in range(year + 1, 2036)], extrapolate=True)

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유'
for yr in range(year + 1, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '경유_예측':pred_list})
die_pred['경유_예측_BSpline'] = spl1pred
die_pred['경유_예측_Akima'] = aki1pred
die_pred

,연도,연료,경유_예측,경유_예측_BSpline,경유_예측_Akima
0,2023,경유,9891328.00,10061712.00,10156376.11
1,2024,경유,9865352.90,10086965.00,10488168.68
2,2025,경유,9839377.80,10112218.00,10890039.05
3,2026,경유,9813402.70,10137471.00,11315943.56
4,2027,경유,9787427.60,10162724.00,11719838.54
5,2028,경유,9761452.50,10187977.00,12055680.35
6,2029,경유,9735477.40,10213230.00,12277425.32
7,2030,경유,9709502.30,10238483.00,12339029.80
8,2031,경유,9683527.20,10263736.00,12194450.12
9,2032,경유,9657552.10,10288989.00,11797642.64


In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유'
for yr in range(year + 1, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '휘발유_예측':pred_list})
gas_pred['휘발유_예측_BSpline'] = spl2pred
gas_pred['휘발유_예측_Akima'] = aki2pred
gas_pred

,연도,연료,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima
0,2023,휘발유,12759346.50,12472513.00,12881770.74
1,2024,휘발유,13160189.70,12915417.00,13505977.46
2,2025,휘발유,13561032.90,13358321.00,14255067.39
3,2026,휘발유,13961876.10,13801225.00,15147441.75
4,2027,휘발유,14362719.30,14244129.00,16201501.77
5,2028,휘발유,14763562.50,14687033.00,17435648.70
6,2029,휘발유,15164405.70,15129937.00,18868283.75
7,2030,휘발유,15565248.90,15572841.00,20517808.17
8,2031,휘발유,15966092.10,16015745.00,22402623.17
9,2032,휘발유,16366935.30,16458649.00,24541130.00


In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG'
for yr in range(year + 1, 2036):
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, 'LPG_예측':pred_list})
lpg_pred['LPG_예측_BSpline'] = spl3pred
lpg_pred['LPG_예측_Akima'] = aki3pred
lpg_pred

,연도,연료,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima
0,2023,LPG,1898243.00,1902560.00,1952460.75
1,2024,LPG,1885073.70,1884782.00,1929016.48
2,2025,LPG,1871904.40,1867004.00,1779271.43
3,2026,LPG,1858735.10,1849226.00,1431104.85
4,2027,LPG,1845565.80,1831448.00,812395.96
5,2028,LPG,1832396.50,1813670.00,-148975.99
6,2029,LPG,1819227.20,1795892.00,-1525131.76
7,2030,LPG,1806057.90,1778114.00,-3388192.11
8,2031,LPG,1792888.60,1760336.00,-5810277.81
9,2032,LPG,1779719.30,1742558.00,-8863509.62


In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)

In [ ]:
die_t = die_t.rename(columns={'차량대수':'경유_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_대수'})

In [ ]:
die_t.columns

Index(['연도', '연료', '경유_대수', '경유_예측', '경유_예측_BSpline', '경유_예측_Akima'], dtype='object')

In [ ]:
df5 = pd.concat([die_t[['연도', '경유_대수', '경유_예측', '경유_예측_BSpline', '경유_예측_Akima']], gas_t[['휘발유_대수', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima']], lpg_t[['LPG_대수', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']]], axis=1)
df5.head()

,연도,경유_대수,경유_예측,경유_예측_BSpline,경유_예측_Akima,휘발유_대수,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima,LPG_대수,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima
0,2019,9985953.00,NaN,NaN,NaN,11143801.00,NaN,NaN,NaN,1955894.00,NaN,NaN,NaN
1,2020,10011206.00,NaN,NaN,NaN,11586705.00,NaN,NaN,NaN,1938116.00,NaN,NaN,NaN
2,2021,9887199.00,NaN,NaN,NaN,11934402.00,NaN,NaN,NaN,1908930.00,NaN,NaN,NaN
3,2022,9940705.00,NaN,NaN,NaN,12364046.00,NaN,NaN,NaN,1921725.00,NaN,NaN,NaN
4,2023,NaN,9891328.00,10061712.00,10156376.11,NaN,12759346.50,12472513.00,12881770.74,NaN,1898243.00,1902560.00,1952460.75


In [ ]:
# 음수 0으로 처리
df5.loc[df5['경유_예측'] < 0, '경유_예측'] = 0
df5.loc[df5['경유_예측_BSpline'] < 0, '경유_예측_BSpline'] = 0
df5.loc[df5['경유_예측_Akima'] < 0, '경유_예측_Akima'] = 0
df5.loc[df5['휘발유_예측'] < 0, '휘발유_예측'] = 0
df5.loc[df5['휘발유_예측_BSpline'] < 0, '휘발유_예측_BSpline'] = 0
df5.loc[df5['휘발유_예측_Akima'] < 0, '휘발유_예측_Akima'] = 0
df5.loc[df5['LPG_예측'] < 0, 'LPG_예측'] = 0
df5.loc[df5['LPG_예측_BSpline'] < 0, 'LPG_예측_BSpline'] = 0
df5.loc[df5['LPG_예측_Akima'] < 0, 'LPG_예측_Akima'] = 0

In [ ]:
# 첫째자리에서 반올림
df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']] = df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']].round(0)

In [ ]:
df5

,연도,경유_대수,경유_예측,경유_예측_BSpline,경유_예측_Akima,휘발유_대수,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima,LPG_대수,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima
0,2019,9985953.00,NaN,NaN,NaN,11143801.00,NaN,NaN,NaN,1955894.00,NaN,NaN,NaN
1,2020,10011206.00,NaN,NaN,NaN,11586705.00,NaN,NaN,NaN,1938116.00,NaN,NaN,NaN
2,2021,9887199.00,NaN,NaN,NaN,11934402.00,NaN,NaN,NaN,1908930.00,NaN,NaN,NaN
3,2022,9940705.00,NaN,NaN,NaN,12364046.00,NaN,NaN,NaN,1921725.00,NaN,NaN,NaN
4,2023,NaN,9891328.00,10061712.00,10156376.00,NaN,12759347.00,12472513.00,12881771.00,NaN,1898243.00,1902560.00,1952461.00
5,2024,NaN,9865353.00,10086965.00,10488169.00,NaN,13160190.00,12915417.00,13505977.00,NaN,1885074.00,1884782.00,1929016.00
6,2025,NaN,9839378.00,10112218.00,10890039.00,NaN,13561033.00,13358321.00,14255067.00,NaN,1871904.00,1867004.00,1779271.00
7,2026,NaN,9813403.00,10137471.00,11315944.00,NaN,13961876.00,13801225.00,15147442.00,NaN,1858735.00,1849226.00,1431105.00
8,2027,NaN,9787428.00,10162724.00,11719839.00,NaN,14362719.00,14244129.00,16201502.00,NaN,1845566.00,1831448.00,812396.00
9,2028,NaN,9761453.00,10187977.00,12055680.00,NaN,14763562.00,14687033.00,17435649.00,NaN,1832397.00,1813670.00,0.00


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df5['테이블생성일자'] = today_date

In [ ]:
list(df5.columns)

['연도',
 '경유_대수',
 '경유_예측',
 '경유_예측_BSpline',
 '경유_예측_Akima',
 '휘발유_대수',
 '휘발유_예측',
 '휘발유_예측_BSpline',
 '휘발유_예측_Akima',
 'LPG_대수',
 'LPG_예측',
 'LPG_예측_BSpline',
 'LPG_예측_Akima',
 '테이블생성일자']

In [ ]:
df5 = df5[[
   '테이블생성일자',
   '연도',
   '경유_대수',
   '휘발유_대수',
   'LPG_대수',
   '경유_예측',
   '경유_예측_BSpline',
   '경유_예측_Akima',
   '휘발유_예측',
   '휘발유_예측_BSpline',
   '휘발유_예측_Akima',
   'LPG_예측',
   'LPG_예측_BSpline',
   'LPG_예측_Akima',
    ]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '경유_대수':'DSL', 
    '휘발유_대수':'GSL', 
    'LPG_대수':'LPG', 
    '경유_예측':'DSL_PRET', 
    '경유_예측_BSpline':'DSL_PRET_BSPLN', 
    '경유_예측_Akima':'DSL_PRET_AKM', 
    '휘발유_예측':'GSL_PRET', 
    '휘발유_예측_BSpline':'GSL_PRET_BSPLN', 
    '휘발유_예측_Akima':'GSL_PRET_AKM', 
    'LPG_예측':'LPG_PRET',
    'LPG_예측_BSpline':'LPG_PRET_BSPLN',
    'LPG_예측_Akima':'LPG_PRET_AKM',
}

In [ ]:
STD_BD_CAR_PRET = df5.rename(columns=cdict)
STD_BD_CAR_PRET.columns

Index(['LOAD_DT', 'YR', 'DSL', 'GSL', 'LPG', 'DSL_PRET', 'DSL_PRET_BSPLN',
       'DSL_PRET_AKM', 'GSL_PRET', 'GSL_PRET_BSPLN', 'GSL_PRET_AKM',
       'LPG_PRET', 'LPG_PRET_BSPLN', 'LPG_PRET_AKM'],
      dtype='object')

In [ ]:
STD_BD_CAR_PRET.head()

,LOAD_DT,YR,DSL,GSL,LPG,DSL_PRET,DSL_PRET_BSPLN,DSL_PRET_AKM,GSL_PRET,GSL_PRET_BSPLN,GSL_PRET_AKM,LPG_PRET,LPG_PRET_BSPLN,LPG_PRET_AKM
0,20230721,2019,9985953.00,11143801.00,1955894.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230721,2020,10011206.00,11586705.00,1938116.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230721,2021,9887199.00,11934402.00,1908930.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230721,2022,9940705.00,12364046.00,1921725.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230721,2023,NaN,NaN,NaN,9891328.00,10061712.00,10156376.00,12759347.00,12472513.00,12881771.00,1898243.00,1902560.00,1952461.00


In [ ]:
STD_BD_CAR_PRET.tail()

,LOAD_DT,YR,DSL,GSL,LPG,DSL_PRET,DSL_PRET_BSPLN,DSL_PRET_AKM,GSL_PRET,GSL_PRET_BSPLN,GSL_PRET_AKM,LPG_PRET,LPG_PRET_BSPLN,LPG_PRET_AKM
12,20230721,2031,NaN,NaN,NaN,9683527.00,10263736.00,12194450.00,15966092.00,16015745.00,22402623.00,1792889.00,1760336.00,0.00
13,20230721,2032,NaN,NaN,NaN,9657552.00,10288989.00,11797643.00,16366935.00,16458649.00,24541130.00,1779719.00,1742558.00,0.00
14,20230721,2033,NaN,NaN,NaN,9631577.00,10314242.00,11102564.00,16767778.00,16901553.00,26951730.00,1766550.00,1724780.00,0.00
15,20230721,2034,NaN,NaN,NaN,9605602.00,10339495.00,10063170.00,17168622.00,17344457.00,29652824.00,1753381.00,1707002.00,0.00
16,20230721,2035,NaN,NaN,NaN,9579627.00,10364748.00,8633417.00,17569465.00,17787361.00,32662814.00,1740211.00,1689224.00,0.00


### [출력] STD_BD_CAR_PRET

In [ ]:
# expdf = STD_BD_CAR_PRET
# table_nm = 'STD_BD_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 
# STD_BD_CAR_PRET.to_csv(os.path.join(df3_fold, 'STD_BD_CAR_PRET.csv'), index=False)

## 하이브리드 연료, 연도별 차량 현황 예측
- 경유 하이브리드, 휘발유 하이브리드, LPG 하이브리드

In [ ]:
dfm2h = dfm2.loc[(dfm2['연료'] == '경유 하이브리드') | (dfm2['연료'] == '휘발유 하이브리드') | (dfm2['연료'] == 'LPG 하이브리드')].reset_index(drop=True)
dfm2h.shape

(1241993, 26)

In [ ]:
errc2h = errc2.loc[(errc2['연료'] == '경유 하이브리드') | (errc2['연료'] == '휘발유 하이브리드') | (errc2['연료'] == 'LPG 하이브리드')].reset_index(drop=True)
errc2h.shape

(17172, 27)

In [ ]:
# 2022년 차량 대수
grp1 = dfm2h.groupby('연료', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', '연료', '차량대수']]

In [ ]:
grp1.head()

,연도,연료,차량대수
0,2022,LPG 하이브리드,14410
1,2022,경유 하이브리드,39484
2,2022,휘발유 하이브리드,1188099


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['연료'].unique():
    for yr in range(2019, year + 1):
        yr_list.append(str(yr))
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list})

In [ ]:
base.head()

,연도,연료
0,2019,LPG 하이브리드
1,2020,LPG 하이브리드
2,2021,LPG 하이브리드
3,2022,LPG 하이브리드
4,2019,경유 하이브리드


In [ ]:
# 연도별 등록대수
grp2 = dfm2h.groupby(['최초등록일자_년', '연료'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,연료,등록대수
0,2004,휘발유 하이브리드,2
1,2005,휘발유 하이브리드,21
2,2006,휘발유 하이브리드,145
3,2007,휘발유 하이브리드,484
4,2008,휘발유 하이브리드,901
5,2009,LPG 하이브리드,4156
6,2009,휘발유 하이브리드,911
7,2010,LPG 하이브리드,4566
8,2010,휘발유 하이브리드,1866
9,2011,LPG 하이브리드,3032


In [ ]:
# 연도별 말소대수
grp3 = errc2h.groupby(['변경일자_년', '연료'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,연료,말소대수
0,2019,LPG 하이브리드,327
1,2019,휘발유 하이브리드,1883
2,2020,LPG 하이브리드,726
3,2020,경유 하이브리드,7
4,2020,휘발유 하이브리드,4323
5,2021,LPG 하이브리드,1067
6,2021,경유 하이브리드,17
7,2021,휘발유 하이브리드,5268
8,2022,LPG 하이브리드,540
9,2022,경유 하이브리드,19


In [ ]:
base1 = base.merge(grp1, on=['연도', '연료'], how='left')
base1

,연도,연료,차량대수
0,2019,LPG 하이브리드,NaN
1,2020,LPG 하이브리드,NaN
2,2021,LPG 하이브리드,NaN
3,2022,LPG 하이브리드,14410.00
4,2019,경유 하이브리드,NaN
5,2020,경유 하이브리드,NaN
6,2021,경유 하이브리드,NaN
7,2022,경유 하이브리드,39484.00
8,2019,휘발유 하이브리드,NaN
9,2020,휘발유 하이브리드,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '연료'], how='left')
base2

,연도,연료,차량대수,등록대수
0,2019,LPG 하이브리드,NaN,1.00
1,2020,LPG 하이브리드,NaN,NaN
2,2021,LPG 하이브리드,NaN,1.00
3,2022,LPG 하이브리드,14410.00,1.00
4,2019,경유 하이브리드,NaN,474.00
5,2020,경유 하이브리드,NaN,7443.00
6,2021,경유 하이브리드,NaN,13813.00
7,2022,경유 하이브리드,39484.00,13620.00
8,2019,휘발유 하이브리드,NaN,102887.00
9,2020,휘발유 하이브리드,NaN,166165.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', '연료'], how='left')
base3

,연도,연료,차량대수,등록대수,말소대수
0,2019,LPG 하이브리드,NaN,1.00,327.00
1,2020,LPG 하이브리드,NaN,NaN,726.00
2,2021,LPG 하이브리드,NaN,1.00,1067.00
3,2022,LPG 하이브리드,14410.00,1.00,540.00
4,2019,경유 하이브리드,NaN,474.00,NaN
5,2020,경유 하이브리드,NaN,7443.00,7.00
6,2021,경유 하이브리드,NaN,13813.00,17.00
7,2022,경유 하이브리드,39484.00,13620.00,19.00
8,2019,휘발유 하이브리드,NaN,102887.00,1883.00
9,2020,휘발유 하이브리드,NaN,166165.00,4323.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    9
등록대수    1
말소대수    1
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,연료,차량대수,등록대수,말소대수
0,2019,LPG 하이브리드,16741.00,1.00,327.00
1,2020,LPG 하이브리드,16015.00,0.00,726.00
2,2021,LPG 하이브리드,14949.00,1.00,1067.00
3,2022,LPG 하이브리드,14410.00,1.00,540.00
4,2019,경유 하이브리드,4651.00,474.00,0.00
5,2020,경유 하이브리드,12087.00,7443.00,7.00
6,2021,경유 하이브리드,25883.00,13813.00,17.00
7,2022,경유 하이브리드,39484.00,13620.00,19.00
8,2019,휘발유 하이브리드,597433.00,102887.00,1883.00
9,2020,휘발유 하이브리드,759275.00,166165.00,4323.00


In [ ]:
die = base3.loc[base3['연료'] == '경유 하이브리드', ['연도', '연료', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['연료'] == '휘발유 하이브리드', ['연도', '연료', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['연료'] == 'LPG 하이브리드', ['연도', '연료', '차량대수']].reset_index(drop=True)

In [ ]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
fit1 = np.polyfit(die['연도'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연도'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연도'], lpg['차량대수'], 1)

In [ ]:
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유 하이브리드'
for yr in range(year + 1, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '경유_하이브리드_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유 하이브리드'
for yr in range(year + 1, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '휘발유_하이브리드_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG 하이브리드'
for yr in range(year + 1, 2036):
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, 'LPG_하이브리드_예측':pred_list})

In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)

In [ ]:
die_t = die_t.rename(columns={'차량대수':'경유_하이브리드_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_하이브리드_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_하이브리드_대수'})

In [ ]:
die_t.columns

Index(['연도', '연료', '경유_하이브리드_대수', '경유_하이브리드_예측'], dtype='object')

In [ ]:
df5 = pd.concat([die_t[['연도', '경유_하이브리드_대수', '경유_하이브리드_예측']], gas_t[['휘발유_하이브리드_대수', '휘발유_하이브리드_예측']], lpg_t[['LPG_하이브리드_대수', 'LPG_하이브리드_예측']]], axis=1)
df5.head()

,연도,경유_하이브리드_대수,경유_하이브리드_예측,휘발유_하이브리드_대수,휘발유_하이브리드_예측,LPG_하이브리드_대수,LPG_하이브리드_예측
0,2019,4651.00,NaN,597433.00,NaN,16741.00,NaN
1,2020,12087.00,NaN,759275.00,NaN,16015.00,NaN
2,2021,25883.00,NaN,979935.00,NaN,14949.00,NaN
3,2022,39484.00,NaN,1188099.00,NaN,14410.00,NaN
4,2023,NaN,50100.00,NaN,1379350.00,NaN,13514.00


In [ ]:
# 첫째자리에서 반올림
df5[['경유_하이브리드_대수', '경유_하이브리드_예측', '휘발유_하이브리드_대수', '휘발유_하이브리드_예측', 'LPG_하이브리드_대수', 'LPG_하이브리드_예측']] = df5[['경유_하이브리드_대수', '경유_하이브리드_예측', '휘발유_하이브리드_대수', '휘발유_하이브리드_예측', 'LPG_하이브리드_대수', 'LPG_하이브리드_예측']].round(0)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df5['테이블생성일자'] = today_date

In [ ]:
df5 = df5[[
    '테이블생성일자', 
    '연도', 
    '휘발유_하이브리드_대수', 
    '경유_하이브리드_대수', 
    'LPG_하이브리드_대수', 
    '휘발유_하이브리드_예측', 
    '경유_하이브리드_예측', 
    'LPG_하이브리드_예측'
    ]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '휘발유_하이브리드_대수':'GSLH', 
    '경유_하이브리드_대수':'DSLH', 
    'LPG_하이브리드_대수':'LPGH', 
    '휘발유_하이브리드_예측':'GSLH_PRET', 
    '경유_하이브리드_예측':'DSLH_PRET', 
    'LPG_하이브리드_예측':'LPGH_PRET',
}

In [ ]:
STD_BD_HYBRD_CAR_PRET = df5.rename(columns=cdict)
STD_BD_HYBRD_CAR_PRET.columns

Index(['LOAD_DT', 'YR', 'GSLH', 'DSLH', 'LPGH', 'GSLH_PRET', 'DSLH_PRET',
       'LPGH_PRET'],
      dtype='object')

In [ ]:
STD_BD_HYBRD_CAR_PRET.head()

,LOAD_DT,YR,GSLH,DSLH,LPGH,GSLH_PRET,DSLH_PRET,LPGH_PRET
0,20230721,2019,597433.00,4651.00,16741.00,NaN,NaN,NaN
1,20230721,2020,759275.00,12087.00,16015.00,NaN,NaN,NaN
2,20230721,2021,979935.00,25883.00,14949.00,NaN,NaN,NaN
3,20230721,2022,1188099.00,39484.00,14410.00,NaN,NaN,NaN
4,20230721,2023,NaN,NaN,NaN,1379350.00,50100.00,13514.00


In [ ]:
STD_BD_HYBRD_CAR_PRET.tail()

,LOAD_DT,YR,GSLH,DSLH,LPGH,GSLH_PRET,DSLH_PRET,LPGH_PRET
12,20230721,2031,NaN,NaN,NaN,2973476.00,144736.00,7067.00
13,20230721,2032,NaN,NaN,NaN,3172742.00,156565.00,6261.00
14,20230721,2033,NaN,NaN,NaN,3372008.00,168395.00,5455.00
15,20230721,2034,NaN,NaN,NaN,3571274.00,180224.00,4649.00
16,20230721,2035,NaN,NaN,NaN,3770540.00,192054.00,3843.00


### [출력] STD_BD_HYBRD_CAR_PRET

In [ ]:
# expdf = STD_BD_HYBRD_CAR_PRET
# table_nm = 'STD_BD_HYBRD_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 
# STD_BD_HYBRD_CAR_PRET.to_csv(os.path.join(df3_fold, 'STD_BD_HYBRD_CAR_PRET.csv'), index=False)

## 내연차 연료, 등급, 연도별 차량 현황 예측
- 경유, 휘발유, LPG

In [ ]:
# 2022년 차량 대수
grp1 = dfm2dgl.groupby(['fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel', '배출가스등급', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,배출가스등급,차량대수
0,2022,LPG,1,464598
1,2022,LPG,2,1176930
2,2022,LPG,3,238483
3,2022,LPG,4,24592
4,2022,LPG,5,436


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['fuel'].unique():
    for grd in grp1['배출가스등급'].unique():
        for yr in range(2019, year + 1):
            yr_list.append(str(yr))
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list})

In [ ]:
base.head()

,연도,fuel,배출가스등급
0,2019,LPG,1
1,2020,LPG,1
2,2021,LPG,1
3,2022,LPG,1
4,2019,LPG,2


In [ ]:
# 연도별 등록대수
grp2 = dfm2dgl.groupby(['최초등록일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,배출가스등급,등록대수
0,1955,경유,5,1
1,1958,휘발유,5,1
2,1962,경유,5,8
3,1962,휘발유,5,1
4,1963,경유,5,2
...,...,...,...,...
442,2023,경유,3,40977
443,2023,경유,X,64014
444,2023,휘발유,1,33131
445,2023,휘발유,2,140120


In [ ]:
# 연도별 말소대수
grp3 = errc2dgl.groupby(['변경일자_년', 'fuel', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,배출가스등급,말소대수
0,2019,LPG,1,5648
1,2019,LPG,2,27864
2,2019,LPG,3,42824
3,2019,LPG,4,3460
4,2019,LPG,5,3
...,...,...,...,...
59,2022,휘발유,2,116667
60,2022,휘발유,3,94401
61,2022,휘발유,4,11550
62,2022,휘발유,5,206


In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel', '배출가스등급'], how='left')
base1

,연도,fuel,배출가스등급,차량대수
0,2019,LPG,1,NaN
1,2020,LPG,1,NaN
2,2021,LPG,1,NaN
3,2022,LPG,1,464598.00
4,2019,LPG,2,NaN
...,...,...,...,...
67,2022,휘발유,5,30011.00
68,2019,휘발유,X,NaN
69,2020,휘발유,X,NaN
70,2021,휘발유,X,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', 'fuel', '배출가스등급'], how='left')
base2

,연도,fuel,배출가스등급,차량대수,등록대수
0,2019,LPG,1,NaN,62978.00
1,2020,LPG,1,NaN,47598.00
2,2021,LPG,1,NaN,36005.00
3,2022,LPG,1,464598.00,16628.00
4,2019,LPG,2,NaN,65541.00
...,...,...,...,...,...
67,2022,휘발유,5,30011.00,NaN
68,2019,휘발유,X,NaN,2595.00
69,2020,휘발유,X,NaN,4517.00
70,2021,휘발유,X,NaN,5502.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', 'fuel', '배출가스등급'], how='left')
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,NaN,62978.00,5648.00
1,2020,LPG,1,NaN,47598.00,15160.00
2,2021,LPG,1,NaN,36005.00,22704.00
3,2022,LPG,1,464598.00,16628.00,9699.00
4,2019,LPG,2,NaN,65541.00,27864.00
...,...,...,...,...,...,...
67,2022,휘발유,5,30011.00,NaN,206.00
68,2019,휘발유,X,NaN,2595.00,298.00
69,2020,휘발유,X,NaN,4517.00,751.00
70,2021,휘발유,X,NaN,5502.00,1919.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    56
등록대수    25
말소대수     8
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,배출가스등급,차량대수,등록대수,말소대수
0,2019,LPG,1,411930.00,62978.00,5648.00
1,2020,LPG,1,444368.00,47598.00,15160.00
2,2021,LPG,1,457669.00,36005.00,22704.00
3,2022,LPG,1,464598.00,16628.00,9699.00
4,2019,LPG,2,1102310.00,65541.00,27864.00
...,...,...,...,...,...,...
67,2022,휘발유,5,30011.00,0.00,206.00
68,2019,휘발유,X,121783.00,2595.00,298.00
69,2020,휘발유,X,125549.00,4517.00,751.00
70,2021,휘발유,X,129132.00,5502.00,1919.00


In [ ]:
die = base3.loc[base3['fuel'] == '경유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
gas = base3.loc[base3['fuel'] == '휘발유', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)
lpg = base3.loc[base3['fuel'] == 'LPG', ['연도', 'fuel', '배출가스등급', '차량대수']].reset_index(drop=True)

In [ ]:
die['연도'] = die['연도'].astype('int')
gas['연도'] = gas['연도'].astype('int')
lpg['연도'] = lpg['연도'].astype('int')

In [ ]:
die['배출가스등급'].unique()

array(['1', '2', '3', '4', '5', 'X'], dtype=object)

In [ ]:
die1 = die.loc[die['배출가스등급'] == '1'].reset_index(drop=True)
die2 = die.loc[die['배출가스등급'] == '2'].reset_index(drop=True)
die3 = die.loc[die['배출가스등급'] == '3'].reset_index(drop=True)
die4 = die.loc[die['배출가스등급'] == '4'].reset_index(drop=True)
die5 = die.loc[die['배출가스등급'] == '5'].reset_index(drop=True)

In [ ]:
gas1 = gas.loc[gas['배출가스등급'] == '1'].reset_index(drop=True)
gas2 = gas.loc[gas['배출가스등급'] == '2'].reset_index(drop=True)
gas3 = gas.loc[gas['배출가스등급'] == '3'].reset_index(drop=True)
gas4 = gas.loc[gas['배출가스등급'] == '4'].reset_index(drop=True)
gas5 = gas.loc[gas['배출가스등급'] == '5'].reset_index(drop=True)

In [ ]:
lpg1 = lpg.loc[lpg['배출가스등급'] == '1'].reset_index(drop=True)
lpg2 = lpg.loc[lpg['배출가스등급'] == '2'].reset_index(drop=True)
lpg3 = lpg.loc[lpg['배출가스등급'] == '3'].reset_index(drop=True)
lpg4 = lpg.loc[lpg['배출가스등급'] == '4'].reset_index(drop=True)
lpg5 = lpg.loc[lpg['배출가스등급'] == '5'].reset_index(drop=True)

In [ ]:
fit_d1 = np.polyfit(die1['연도'], die1['차량대수'], 1)
fit_d2 = np.polyfit(die2['연도'], die2['차량대수'], 1)
fit_d3 = np.polyfit(die3['연도'], die3['차량대수'], 1)
fit_d4 = np.polyfit(die4['연도'], die4['차량대수'], 1)
fit_d5 = np.polyfit(die5['연도'], die5['차량대수'], 1)

In [ ]:
fit_g1 = np.polyfit(gas1['연도'], gas1['차량대수'], 1)
fit_g2 = np.polyfit(gas2['연도'], gas2['차량대수'], 1)
fit_g3 = np.polyfit(gas3['연도'], gas3['차량대수'], 1)
fit_g4 = np.polyfit(gas4['연도'], gas4['차량대수'], 1)
fit_g5 = np.polyfit(gas5['연도'], gas5['차량대수'], 1)

In [ ]:
fit_l1 = np.polyfit(lpg1['연도'], lpg1['차량대수'], 1)
fit_l2 = np.polyfit(lpg2['연도'], lpg2['차량대수'], 1)
fit_l3 = np.polyfit(lpg3['연도'], lpg3['차량대수'], 1)
fit_l4 = np.polyfit(lpg4['연도'], lpg4['차량대수'], 1)
fit_l5 = np.polyfit(lpg5['연도'], lpg5['차량대수'], 1)

In [ ]:
ad1, bd1 = fit_d1
ad2, bd2 = fit_d2
ad3, bd3 = fit_d3
ad4, bd4 = fit_d4
ad5, bd5 = fit_d5

In [ ]:
ag1, bg1 = fit_g1
ag2, bg2 = fit_g2
ag3, bg3 = fit_g3
ag4, bg4 = fit_g4
ag5, bg5 = fit_g5

In [ ]:
al1, bl1 = fit_l1
al2, bl2 = fit_l2
al3, bl3 = fit_l3
al4, bl4 = fit_l4
al5, bl5 = fit_l5

In [ ]:
# 경유 1등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '1'
for yr in range(year + 1, 2036):
    pred = ad1 * yr + bd1
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '2'
for yr in range(year + 1, 2036):
    pred = ad2 * yr + bd2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '3'
for yr in range(year + 1, 2036):
    pred = ad3 * yr + bd3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '4'
for yr in range(year + 1, 2036):
    pred = ad4 * yr + bd4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})
# 경유 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '경유'
grd = '5'
for yr in range(year + 1, 2036):
    pred = ad5 * yr + bd5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
die5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'경유_예측':pred_list})

In [ ]:
# 휘발유 1등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '1'
for yr in range(year + 1, 2036):
    pred = ag1 * yr + bg1
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '2'
for yr in range(year + 1, 2036):
    pred = ag2 * yr + bg2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '3'
for yr in range(year + 1, 2036):
    pred = ag3 * yr + bg3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '4'
for yr in range(year + 1, 2036):
    pred = ag4 * yr + bg4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})
# 휘발유 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = '휘발유'
grd = '5'
for yr in range(year + 1, 2036):
    pred = ag5 * yr + bg5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
gas5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'휘발유_예측':pred_list})

In [ ]:
# LPG 1등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '1'
for yr in range(year + 1, 2036):
    pred = al1 * yr + bl1
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg1_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 2등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '2'
for yr in range(year + 1, 2036):
    pred = al2 * yr + bl2
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg2_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 3등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '3'
for yr in range(year + 1, 2036):
    pred = al3 * yr + bl3
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg3_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 4등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '4'
for yr in range(year + 1, 2036):
    pred = al4 * yr + bl4
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg4_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})
# LPG 5등급 예측
yr_list = []
fuel_list = []
grd_list = []
pred_list = []
fuel = 'LPG'
grd = '5'
for yr in range(year + 1, 2036):
    pred = al5 * yr + bl5
    yr_list.append(yr)
    fuel_list.append(fuel)
    grd_list.append(grd)
    pred_list.append(pred)
lpg5_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '배출가스등급':grd_list,'LPG_예측':pred_list})

In [ ]:
die1.head()

,연도,fuel,배출가스등급,차량대수
0,2019,경유,1,0.00
1,2020,경유,1,0.00
2,2021,경유,1,0.00
3,2022,경유,1,0.00


In [ ]:
die_t = pd.concat([die1, die1_pred, die2, die2_pred, die3, die3_pred, die4, die4_pred, die5, die5_pred], ignore_index=True)
gas_t = pd.concat([gas1, gas1_pred, gas2, gas2_pred, gas3, gas3_pred, gas4, gas4_pred, gas5, gas5_pred], ignore_index=True)
lpg_t = pd.concat([lpg1, lpg1_pred, lpg2, lpg2_pred, lpg3, lpg3_pred, lpg4, lpg4_pred, lpg5, lpg5_pred], ignore_index=True)

In [ ]:
die_t = die_t.rename(columns={'경유_예측':'차량예측'})
gas_t = gas_t.rename(columns={'휘발유_예측':'차량예측'})
lpg_t = lpg_t.rename(columns={'LPG_예측':'차량예측'})

In [ ]:
df6 = pd.concat([die_t, gas_t, lpg_t], ignore_index=True)

In [ ]:
df6.loc[df6['차량예측'] < 0, '차량예측'] = 0

In [ ]:
df6.columns

Index(['연도', 'fuel', '배출가스등급', '차량대수', '차량예측'], dtype='object')

In [ ]:
df6[['차량대수', '차량예측']] = df6[['차량대수', '차량예측']].round(0)

In [ ]:
df6['테이블생성일자'] = today_date

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    'fuel':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT', 
    '차량예측':'VHCL_PRET', 
}

In [ ]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET = df6.rename(columns=cdict)
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.columns

Index(['YR', 'FUEL_CD', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT', 'VHCL_PRET',
       'LOAD_DT'],
      dtype='object')

In [ ]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.head()

,YR,FUEL_CD,EXHST_GAS_GRD_CD,VHCL_MKCNT,VHCL_PRET,LOAD_DT
0,2019,경유,1,0.00,NaN,20230721
1,2020,경유,1,0.00,NaN,20230721
2,2021,경유,1,0.00,NaN,20230721
3,2022,경유,1,0.00,NaN,20230721
4,2023,경유,1,NaN,0.00,20230721


### [출력] STD_BD_FUEL_GRD_VHCL_CURSTT_PRET

In [ ]:
# expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
# table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 
# STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.to_csv(os.path.join(df3_fold, 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.csv'), index=False)

## 무공해차 연료, 연도별 차량 현황 예측
- 전기, 수소

In [ ]:
dfm2bh = dfm2.loc[(dfm2['fuel'] == '전기') | (dfm2['fuel'] == '수소')].reset_index(drop=True)
dfm2bh.shape

(437341, 26)

In [ ]:
errc2bh = errc2.loc[(errc2['fuel'] == '전기') | (errc2['fuel'] == '수소')].reset_index(drop=True)
errc2bh.shape

(8112, 27)

In [ ]:
# 2022년 차량 대수
grp1 = dfm2bh.groupby('fuel', as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1 = grp1[['연도', 'fuel', '차량대수']]

In [ ]:
grp1.head()

,연도,fuel,차량대수
0,2022,수소,29489
1,2022,전기,407852


In [ ]:
# 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
for fuel in grp1['fuel'].unique():
    for yr in range(2019, year + 1):
        yr_list.append(str(yr))
        fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list})

In [ ]:
base.head()

,연도,fuel
0,2019,수소
1,2020,수소
2,2021,수소
3,2022,수소
4,2019,전기


In [ ]:
# 연도별 등록대수
grp2 = dfm2bh.groupby(['최초등록일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,fuel,등록대수
0,2010,전기,10
1,2011,전기,106
2,2012,전기,369
3,2013,전기,489
4,2014,수소,5
5,2014,전기,1175
6,2015,수소,33
7,2015,전기,2767
8,2016,수소,45
9,2016,전기,3994


In [ ]:
# 연도별 말소대수
grp3 = errc2bh.groupby(['변경일자_년', 'fuel'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,fuel,말소대수
0,2019,수소,5
1,2019,전기,593
2,2020,수소,19
3,2020,전기,1680
4,2021,수소,38
5,2021,전기,3950
6,2022,수소,37
7,2022,전기,1790


In [ ]:
base1 = base.merge(grp1, on=['연도', 'fuel'], how='left')
base1

,연도,fuel,차량대수
0,2019,수소,NaN
1,2020,수소,NaN
2,2021,수소,NaN
3,2022,수소,29489.00
4,2019,전기,NaN
5,2020,전기,NaN
6,2021,전기,NaN
7,2022,전기,407852.00


In [ ]:
base2 = base1.merge(grp2, on=['연도', 'fuel'], how='left')
base2

,연도,fuel,차량대수,등록대수
0,2019,수소,NaN,4163
1,2020,수소,NaN,5812
2,2021,수소,NaN,8516
3,2022,수소,29489.00,8136
4,2019,전기,NaN,33315
5,2020,전기,NaN,46263
6,2021,전기,NaN,100262
7,2022,전기,407852.00,128997


In [ ]:
base3 = base2.merge(grp3, on=['연도', 'fuel'], how='left')
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,수소,NaN,4163,5
1,2020,수소,NaN,5812,19
2,2021,수소,NaN,8516,38
3,2022,수소,29489.00,8136,37
4,2019,전기,NaN,33315,593
5,2020,전기,NaN,46263,1680
6,2021,전기,NaN,100262,3950
7,2022,전기,407852.00,128997,1790


In [ ]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    6
등록대수    0
말소대수    0
dtype: int64

In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,fuel,차량대수,등록대수,말소대수
0,2019,수소,7119.00,4163,5
1,2020,수소,12912.00,5812,19
2,2021,수소,21390.00,8516,38
3,2022,수소,29489.00,8136,37
4,2019,전기,139750.00,33315,593
5,2020,전기,184333.00,46263,1680
6,2021,전기,280645.00,100262,3950
7,2022,전기,407852.00,128997,1790


In [ ]:
bt = base3.loc[base3['fuel'] == '전기', ['연도', 'fuel', '차량대수']].reset_index(drop=True)
hy = base3.loc[base3['fuel'] == '수소', ['연도', 'fuel', '차량대수']].reset_index(drop=True)

In [ ]:
bt['연도'] = bt['연도'].astype('int')
hy['연도'] = hy['연도'].astype('int')

In [ ]:
fit1 = np.polyfit(bt['연도'], bt['차량대수'], 1)
fit2 = np.polyfit(hy['연도'], hy['차량대수'], 1)

In [ ]:
a1, b1 = fit1
a2, b2 = fit2

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '전기'
for yr in range(year + 1, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
bt_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '전기_예측':pred_list})

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '수소'
for yr in range(year + 1, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
hy_pred = pd.DataFrame({'연도':yr_list, 'fuel':fuel_list, '수소_예측':pred_list})

In [ ]:
bt_t = pd.concat([bt, bt_pred], ignore_index=True)
hy_t = pd.concat([hy, hy_pred], ignore_index=True)

In [ ]:
bt_t = bt_t.rename(columns={'차량대수':'전기_대수'})
hy_t = hy_t.rename(columns={'차량대수':'수소_대수'})

In [ ]:
hy_t.columns

Index(['연도', 'fuel', '수소_대수', '수소_예측'], dtype='object')

In [ ]:
df7 = pd.concat([ bt_t[['연도', '전기_대수', '전기_예측']], hy_t[['수소_대수', '수소_예측']] ], axis=1)

In [ ]:
df7

,연도,전기_대수,전기_예측,수소_대수,수소_예측
0,2019,139750.00,NaN,7119.00,NaN
1,2020,184333.00,NaN,12912.00,NaN
2,2021,280645.00,NaN,21390.00,NaN
3,2022,407852.00,NaN,29489.00,NaN
4,2023,NaN,478299.50,NaN,36624.50
5,2024,NaN,568361.30,NaN,44183.30
6,2025,NaN,658423.10,NaN,51742.10
7,2026,NaN,748484.90,NaN,59300.90
8,2027,NaN,838546.70,NaN,66859.70
9,2028,NaN,928608.50,NaN,74418.50


In [ ]:
df7.columns

Index(['연도', '전기_대수', '전기_예측', '수소_대수', '수소_예측'], dtype='object')

In [ ]:
df7[['전기_대수', '전기_예측', '수소_대수', '수소_예측']] = df7[['전기_대수', '전기_예측', '수소_대수', '수소_예측']].round(0)

In [ ]:
df7.columns

Index(['연도', '전기_대수', '전기_예측', '수소_대수', '수소_예측'], dtype='object')

In [ ]:
df7['테이블생성일자'] = today_date

In [ ]:
df7 = df7[[
    '테이블생성일자', 
    '연도', 
    '전기_대수', 
    '수소_대수', 
    '전기_예측',
    '수소_예측', 
    ]]

In [ ]:
cdict = {
    '테이블생성일자':'LOAD_DT', 
    '연도':'YR', 
    '전기_대수':'BTYCR', 
    '수소_대수':'HY', 
    '전기_예측':'BTYCR_PRET',
    '수소_예측':'HY_PRET',
}

In [ ]:
STD_BD_ECO_CAR_PRET = df7.rename(columns=cdict)
STD_BD_ECO_CAR_PRET.columns

Index(['LOAD_DT', 'YR', 'BTYCR', 'HY', 'BTYCR_PRET', 'HY_PRET'], dtype='object')

### [출력] STD_BD_ECO_CAR_PRET

In [ ]:
# expdf = STD_BD_ECO_CAR_PRET
# table_nm = 'STD_BD_ECO_CAR_PRET'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [ ]:
# # 
# STD_BD_ECO_CAR_PRET.to_csv(os.path.join(df3_fold, 'STD_BD_ECO_CAR_PRET.csv'), index=False)

## 지역, 등급별 말소 차량 현황

In [56]:
errc2.shape

(3606820, 27)

In [57]:
errc2.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자', '시도', '시군구', 'fuel', '지역', '변경일자_년',
       '변경일자_월', '변경일자_일', '시군구_수정'],
      dtype='object')

In [60]:
errc2['테이블생성일자'] = today_date

In [61]:
df8 = errc2[[
    '테이블생성일자', 
    '법정동코드',
    '시도',
    '시군구', 
    '차대번호', 
    '변경일자', 
    '배출가스등급', 
    '연료'
    ]]
df8.columns

Index(['테이블생성일자', '법정동코드', '시도', '시군구', '차대번호', '변경일자', '배출가스등급', '연료'], dtype='object')

In [62]:
df8['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '휘발유 하이브리드', '전기', 'LPG 하이브리드', '수소',
       '경유 하이브리드'], dtype=object)

In [63]:
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '법정동코드':'STDG_CD',
    '시도':'CTPV',
    '시군구':'SGG', 
    '차대번호':'VIN', 
    '변경일자':'CHG_YMD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '연료':'FUEL_CD', 
}

In [64]:
STD_BD_ERSR_RSLT = df8.rename(columns=chc_col)
STD_BD_ERSR_RSLT.columns

Index(['LOAD_DT', 'STDG_CD', 'CTPV', 'SGG', 'VIN', 'CHG_YMD',
       'EXHST_GAS_GRD_CD', 'FUEL_CD'],
      dtype='object')

### [출력] STD_BD_ERSR_RSLT

In [65]:
expdf = STD_BD_ERSR_RSLT
table_nm = 'STD_BD_ERSR_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

In [66]:
# 12.9s
STD_BD_ERSR_RSLT.to_csv(os.path.join(df3_fold, 'STD_BD_ERSR_RSLT.csv'), index=False)

## 등급별현황 테이블
- 시도, 연도, 월, 등급, 연료, 차종, 차량유형, 용도별 / 차량대수, 말소차량대수, 차량 비율

In [284]:
df9 = dfm.copy()

In [285]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230721'

In [286]:
# 데이터 연도 설정
year = 2022
month = 12
# year = int(today_date[:4])
# month = int(today_date[4:6])

In [287]:
df9.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '시도', '시군구', 'fuel', '지역', '최초등록일자_년', '최초등록일자_월', '최초등록일자_일',
       '시군구_수정'],
      dtype='object')

In [288]:
# 2022년 차량 대수
grp1 = df9.groupby(['시도', '배출가스등급', '연료', '차종', '차종유형', '용도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = f'{year}'
grp1['월'] = f'{month}'
grp1

,시도,배출가스등급,연료,차종,차종유형,용도,차량대수,연도,월
0,강원도,1,LPG 하이브리드,승용,중형,개인용,572,2022,12
1,강원도,1,LPG 하이브리드,승용,중형,관용,25,2022,12
2,강원도,1,LPG(액화석유가스),승용,중형,개인용,7666,2022,12
3,강원도,1,LPG(액화석유가스),승용,중형,관용,7,2022,12
4,강원도,1,LPG(액화석유가스),승용,중형,영업용,5628,2022,12
...,...,...,...,...,...,...,...,...,...
5619,충청북도,X,휘발유 하이브리드,승용,대형,개인용,164,2022,12
5620,충청북도,X,휘발유 하이브리드,승용,대형,영업용,10,2022,12
5621,충청북도,X,휘발유 하이브리드,승용,중형,개인용,1648,2022,12
5622,충청북도,X,휘발유 하이브리드,승용,중형,관용,1,2022,12


In [289]:
date_date = '20221231'
# date_date = datetime.today().strftime("%Y%m%d")

In [290]:
list(pd.date_range(end=date_date, periods=4, freq='MS').month)

[9, 10, 11, 12]

In [291]:
# 37.5s
# 차량 통계 기본 데이터셋
ctpv_list, yr_list, month_list, grd_list, fuel_list, vhcty_list, ty_list, purps_list = [], [], [], [], [], [], [], []
for ctpv in grp1['시도'].unique():
    for yr in grp1['연도'].unique():
        for month in list(pd.date_range(end='20221231', periods=4, freq='MS').month):
            for grd in grp1['배출가스등급'].unique():
                for fuel in grp1['연료'].unique():
                    for vhcty in grp1['차종'].unique():
                        for ty in grp1['차종유형'].unique():
                            for purps in grp1['용도'].unique():
                                ctpv_list.append(ctpv)
                                yr_list.append(str(yr))
                                month_list.append(f'{month:0>2}')
                                grd_list.append(grd)
                                fuel_list.append(fuel)
                                vhcty_list.append(vhcty)
                                ty_list.append(ty)
                                purps_list.append(purps)
base = pd.DataFrame({
    '시도':ctpv_list, 
    '연도':yr_list, 
    '월':month_list, 
    '배출가스등급':grd_list, 
    '연료':fuel_list, 
    '차종':vhcty_list, 
    '차종유형':ty_list, 
    '용도':purps_list, 
    })

In [292]:
base.head()

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도
0,강원도,2022,09,1,LPG 하이브리드,승용,중형,개인용
1,강원도,2022,09,1,LPG 하이브리드,승용,중형,관용
2,강원도,2022,09,1,LPG 하이브리드,승용,중형,영업용
3,강원도,2022,09,1,LPG 하이브리드,승용,경형,개인용
4,강원도,2022,09,1,LPG 하이브리드,승용,경형,관용


In [293]:
# 13.6s
# 연도별 등록대수
grp2 = df9.groupby(['시도', '최초등록일자_년', '최초등록일자_월', '배출가스등급', '연료', '차종', '차종유형', '용도'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '최초등록일자_월':'월', '차대번호':'등록대수'})
grp2

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,등록대수
0,강원도,1955,06,5,경유,화물,중형,개인용,1
1,강원도,1962,06,5,경유,화물,대형,개인용,2
2,강원도,1962,06,5,경유,화물,중형,영업용,1
3,강원도,1962,08,5,경유,화물,중형,개인용,1
4,강원도,1963,09,5,경유,화물,중형,영업용,1
...,...,...,...,...,...,...,...,...,...
274127,충청북도,2023,06,X,휘발유,승용,중형,개인용,188
274128,충청북도,2023,06,X,휘발유,승용,중형,영업용,3
274129,충청북도,2023,06,X,휘발유 하이브리드,승용,대형,개인용,4
274130,충청북도,2023,06,X,휘발유 하이브리드,승용,대형,영업용,2


In [294]:
#2.5s
# 연도별 말소대수
grp3 = errc.groupby(['시도', '변경일자_년', '변경일자_월', '배출가스등급', '연료', '차종', '차종유형', '용도'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '변경일자_월':'월', '차대번호':'말소대수'})
grp3

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,말소대수
0,강원도,2019,04,4,경유,화물,소형,개인용,1
1,강원도,2019,06,1,LPG 하이브리드,승용,중형,개인용,1
2,강원도,2019,06,1,LPG(액화석유가스),승용,중형,영업용,3
3,강원도,2019,06,2,LPG(액화석유가스),승용,경형,개인용,3
4,강원도,2019,06,2,LPG(액화석유가스),승용,대형,개인용,1
...,...,...,...,...,...,...,...,...,...
58118,충청북도,2022,06,5,경유,화물,중형,관용,2
58119,충청북도,2022,06,5,경유,화물,중형,영업용,5
58120,충청북도,2022,06,5,휘발유,승용,중형,개인용,2
58121,충청북도,2022,06,X,경유,승용,중형,개인용,1


In [295]:
base1 = base.merge(grp1, on=['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도'], how='left')
base1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수
0,강원도,2022,09,1,LPG 하이브리드,승용,중형,개인용,NaN
1,강원도,2022,09,1,LPG 하이브리드,승용,중형,관용,NaN
2,강원도,2022,09,1,LPG 하이브리드,승용,중형,영업용,NaN
3,강원도,2022,09,1,LPG 하이브리드,승용,경형,개인용,NaN
4,강원도,2022,09,1,LPG 하이브리드,승용,경형,관용,NaN
...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,등유,화물,소형,관용,NaN
293756,충청북도,2022,12,X,등유,화물,소형,영업용,NaN
293757,충청북도,2022,12,X,등유,화물,대형,개인용,NaN
293758,충청북도,2022,12,X,등유,화물,대형,관용,NaN


In [296]:
base2 = base1.merge(grp2, on=['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도'], how='left')
base2

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수
0,강원도,2022,09,1,LPG 하이브리드,승용,중형,개인용,NaN,NaN
1,강원도,2022,09,1,LPG 하이브리드,승용,중형,관용,NaN,NaN
2,강원도,2022,09,1,LPG 하이브리드,승용,중형,영업용,NaN,NaN
3,강원도,2022,09,1,LPG 하이브리드,승용,경형,개인용,NaN,NaN
4,강원도,2022,09,1,LPG 하이브리드,승용,경형,관용,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,등유,화물,소형,관용,NaN,NaN
293756,충청북도,2022,12,X,등유,화물,소형,영업용,NaN,NaN
293757,충청북도,2022,12,X,등유,화물,대형,개인용,NaN,NaN
293758,충청북도,2022,12,X,등유,화물,대형,관용,NaN,NaN


In [297]:
base3 = base2.merge(grp3, on=['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도'], how='left')
base3

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
0,강원도,2022,09,1,LPG 하이브리드,승용,중형,개인용,NaN,NaN,NaN
1,강원도,2022,09,1,LPG 하이브리드,승용,중형,관용,NaN,NaN,NaN
2,강원도,2022,09,1,LPG 하이브리드,승용,중형,영업용,NaN,NaN,NaN
3,강원도,2022,09,1,LPG 하이브리드,승용,경형,개인용,NaN,NaN,NaN
4,강원도,2022,09,1,LPG 하이브리드,승용,경형,관용,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,등유,화물,소형,관용,NaN,NaN,NaN
293756,충청북도,2022,12,X,등유,화물,소형,영업용,NaN,NaN,NaN
293757,충청북도,2022,12,X,등유,화물,대형,개인용,NaN,NaN,NaN
293758,충청북도,2022,12,X,등유,화물,대형,관용,NaN,NaN,NaN


In [298]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    288136
등록대수    287974
말소대수    293760
dtype: int64

In [299]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [300]:
base3 = base3.sort_values(['시도', '배출가스등급', '연료', '차종', '차종유형', '용도', '연도', '월']).reset_index(drop=True)
base3

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
0,강원도,2022,09,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,0.00
1,강원도,2022,10,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,0.00
2,강원도,2022,11,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,0.00
3,강원도,2022,12,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,0.00
4,강원도,2022,09,1,CNG 하이브리드,승용,경형,관용,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,휘발유 하이브리드,화물,중형,관용,0.00,0.00,0.00
293756,충청북도,2022,09,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,0.00
293757,충청북도,2022,10,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,0.00
293758,충청북도,2022,11,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,0.00


In [301]:
base3[base3['차량대수'] != 0]

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
615,강원도,2022,12,1,LPG 하이브리드,승용,중형,개인용,572.00,0.00,0.00
619,강원도,2022,12,1,LPG 하이브리드,승용,중형,관용,25.00,0.00,0.00
807,강원도,2022,12,1,LPG(액화석유가스),승용,중형,개인용,7666.00,1.00,0.00
811,강원도,2022,12,1,LPG(액화석유가스),승용,중형,관용,7.00,0.00,0.00
815,강원도,2022,12,1,LPG(액화석유가스),승용,중형,영업용,5628.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
293583,충청북도,2022,12,X,휘발유 하이브리드,승용,대형,개인용,164.00,25.00,0.00
293591,충청북도,2022,12,X,휘발유 하이브리드,승용,대형,영업용,10.00,1.00,0.00
293607,충청북도,2022,12,X,휘발유 하이브리드,승용,중형,개인용,1648.00,123.00,0.00
293611,충청북도,2022,12,X,휘발유 하이브리드,승용,중형,관용,1.00,0.00,0.00


In [302]:
base3.iloc[293607-3:293607+1]

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
293604,충청북도,2022,09,X,휘발유 하이브리드,승용,중형,개인용,0.00,161.00,0.00
293605,충청북도,2022,10,X,휘발유 하이브리드,승용,중형,개인용,0.00,34.00,0.00
293606,충청북도,2022,11,X,휘발유 하이브리드,승용,중형,개인용,0.00,54.00,0.00
293607,충청북도,2022,12,X,휘발유 하이브리드,승용,중형,개인용,1648.00,123.00,0.00


In [303]:
# 1m 28.6s
n = len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [304]:
base3.iloc[293607-3:293607+1]

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,등록대수,말소대수
293604,충청북도,2022,09,X,휘발유 하이브리드,승용,중형,개인용,1437.00,161.00,0.00
293605,충청북도,2022,10,X,휘발유 하이브리드,승용,중형,개인용,1471.00,34.00,0.00
293606,충청북도,2022,11,X,휘발유 하이브리드,승용,중형,개인용,1525.00,54.00,0.00
293607,충청북도,2022,12,X,휘발유 하이브리드,승용,중형,개인용,1648.00,123.00,0.00


In [305]:
anl1 = base3[['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도', '차량대수', '말소대수']]
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수
0,강원도,2022,09,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00
1,강원도,2022,10,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00
2,강원도,2022,11,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00
3,강원도,2022,12,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00
4,강원도,2022,09,1,CNG 하이브리드,승용,경형,관용,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,휘발유 하이브리드,화물,중형,관용,0.00,0.00
293756,충청북도,2022,09,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00
293757,충청북도,2022,10,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00
293758,충청북도,2022,11,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00


In [306]:
anl1.columns

Index(['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도', '차량대수', '말소대수'], dtype='object')

In [307]:
anl1.groupby(['연도'])['차량대수'].transform('sum')

0        103417102.00
1        103417102.00
2        103417102.00
3        103417102.00
4        103417102.00
             ...     
293755   103417102.00
293756   103417102.00
293757   103417102.00
293758   103417102.00
293759   103417102.00
Name: 차량대수, Length: 293760, dtype: float64

In [308]:
anl1.groupby(['연도', '연료'])['차량대수'].transform('sum')

0           1216.00
1           1216.00
2           1216.00
3           1216.00
4           1216.00
            ...    
293755   4694919.00
293756   4694919.00
293757   4694919.00
293758   4694919.00
293759   4694919.00
Name: 차량대수, Length: 293760, dtype: float64

In [309]:
anl1['연도별_차량대수'] = anl1.groupby(['연도'])['차량대수'].transform('sum')
anl1['연도_연료별_차량대수'] = anl1.groupby(['연도', '연료'])['차량대수'].transform('sum')

In [310]:
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수,연도별_차량대수,연도_연료별_차량대수
0,강원도,2022,09,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00
1,강원도,2022,10,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00
2,강원도,2022,11,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00
3,강원도,2022,12,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00
4,강원도,2022,09,1,CNG 하이브리드,승용,경형,관용,0.00,0.00,103417102.00,1216.00
...,...,...,...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,휘발유 하이브리드,화물,중형,관용,0.00,0.00,103417102.00,4694919.00
293756,충청북도,2022,09,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,103417102.00,4694919.00
293757,충청북도,2022,10,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,103417102.00,4694919.00
293758,충청북도,2022,11,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,103417102.00,4694919.00


In [311]:
anl1['연도_연료차량비율'] = anl1['연도_연료별_차량대수'] / anl1['연도별_차량대수']
anl1

,시도,연도,월,배출가스등급,연료,차종,차종유형,용도,차량대수,말소대수,연도별_차량대수,연도_연료별_차량대수,연도_연료차량비율
0,강원도,2022,09,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00,0.00
1,강원도,2022,10,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00,0.00
2,강원도,2022,11,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00,0.00
3,강원도,2022,12,1,CNG 하이브리드,승용,경형,개인용,0.00,0.00,103417102.00,1216.00,0.00
4,강원도,2022,09,1,CNG 하이브리드,승용,경형,관용,0.00,0.00,103417102.00,1216.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293755,충청북도,2022,12,X,휘발유 하이브리드,화물,중형,관용,0.00,0.00,103417102.00,4694919.00,0.05
293756,충청북도,2022,09,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,103417102.00,4694919.00,0.05
293757,충청북도,2022,10,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,103417102.00,4694919.00,0.05
293758,충청북도,2022,11,X,휘발유 하이브리드,화물,중형,영업용,0.00,0.00,103417102.00,4694919.00,0.05


In [312]:
anl1['연도_연료차량비율'].dtype

dtype('float64')

In [313]:
anl1.columns

Index(['시도', '연도', '월', '배출가스등급', '연료', '차종', '차종유형', '용도', '차량대수', '말소대수',
       '연도별_차량대수', '연도_연료별_차량대수', '연도_연료차량비율'],
      dtype='object')

In [314]:
today_date

'20230721'

In [315]:
anl1['테이블생성일자'] = today_date

In [316]:
anl1.columns.to_list()

['시도',
 '연도',
 '월',
 '배출가스등급',
 '연료',
 '차종',
 '차종유형',
 '용도',
 '차량대수',
 '말소대수',
 '연도별_차량대수',
 '연도_연료별_차량대수',
 '연도_연료차량비율',
 '테이블생성일자']

In [317]:
STD_BD_DAT_GRD_CURSTT = anl1[[
    '시도',
    '연도',
    '월',
    '배출가스등급',
    '연료',
    '차종',
    '차종유형',
    '용도',
    '차량대수',
    '말소대수',
    '연도_연료차량비율',
    '테이블생성일자'
    ]]
STD_BD_DAT_GRD_CURSTT.shape

(293760, 12)

In [318]:
STD_BD_DAT_GRD_CURSTT.columns.to_list()

['시도',
 '연도',
 '월',
 '배출가스등급',
 '연료',
 '차종',
 '차종유형',
 '용도',
 '차량대수',
 '말소대수',
 '연도_연료차량비율',
 '테이블생성일자']

In [319]:
cdict = {
    '시도':'CTPV', 
    '연도':'YR', 
    '월':'MM', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '연료':'FUEL_CD', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    '용도':'PURPS_CD2', 
    '차량대수':'VHCL_MKCNT', 
    '말소대수':'ERSR_MKCNT', 
    '연도_연료차량비율':'YR_FUEL_VHCL_RT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD_CURSTT = STD_BD_DAT_GRD_CURSTT.rename(columns=cdict)
STD_BD_DAT_GRD_CURSTT.columns

Index(['CTPV', 'YR', 'MM', 'EXHST_GAS_GRD_CD', 'FUEL_CD', 'VHCTY_CD',
       'VHCTY_TY', 'PURPS_CD2', 'VHCL_MKCNT', 'ERSR_MKCNT', 'YR_FUEL_VHCL_RT',
       'LOAD_DT'],
      dtype='object')

In [320]:
STD_BD_DAT_GRD_CURSTT.columns.to_list()

['CTPV',
 'YR',
 'MM',
 'EXHST_GAS_GRD_CD',
 'FUEL_CD',
 'VHCTY_CD',
 'VHCTY_TY',
 'PURPS_CD2',
 'VHCL_MKCNT',
 'ERSR_MKCNT',
 'YR_FUEL_VHCL_RT',
 'LOAD_DT']

### [출력] STD_BD_DAT_GRD_CURSTT

In [321]:
# expdf = STD_BD_DAT_GRD_CURSTT
# table_nm = 'STD_BD_DAT_GRD_CURSTT'.upper()

# # 테이블 생성
# try:
#     sql = 'create or replace table ' + table_nm + '( \n'

#     for idx,column in enumerate(expdf.columns):
#         if 'float' in expdf[column].dtype.name:
#             sql += column + ' float'
#         elif 'int' in expdf[column].dtype.name:
#             sql += column + ' number'
#         else:
#             sql += column + ' varchar(255)'

#         if len(expdf.columns) - 1 != idx:
#             sql += ','
#         sql += '\n'
#     sql += ')'    
#     we.execute(sql)
#     we.import_from_pandas(expdf, table_nm)
# except:
#     # 데이터 추가
#     # 7s
#     we.import_from_pandas(expdf, table_nm)

In [322]:
# # 1.3s
# STD_BD_DAT_GRD_CURSTT.to_csv(os.path.join(df3_fold, 'STD_BD_DAT_GRD_CURSTT.csv'), index=False)

## 내연기관차 감소추이

In [375]:
dfm2dgl.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '시도', '시군구', 'fuel', '지역', '최초등록일자_년', '최초등록일자_월', '최초등록일자_일',
       '시군구_수정'],
      dtype='object')

In [376]:
errc2dgl.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자', '시도', '시군구', 'fuel', '지역', '변경일자_년',
       '변경일자_월', '변경일자_일', '시군구_수정'],
      dtype='object')

In [440]:
today_date

'20230721'

In [467]:
grp1 = dfm2dgl.groupby(['배출가스등급', '연료'])['차대번호'].count().reset_index()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
year = '2022'
month = '12'
# year = today_date[:4]
# month = today_date[4:6]
grp1[['연도', '월']] = [year, month]
grp1

,배출가스등급,연료,차량대수,연도,월
0,1,LPG(액화석유가스),464598,2022,12
1,1,휘발유,995195,2022,12
2,2,LPG(액화석유가스),1176930,2022,12
3,2,휘발유,9992449,2022,12
4,3,LPG(액화석유가스),238483,2022,12
5,3,경유,7617062,2022,12
6,3,휘발유,905591,2022,12
7,4,LPG(액화석유가스),24592,2022,12
8,4,경유,1156086,2022,12
9,4,휘발유,267899,2022,12


In [509]:
grp1['연도'].unique(), grp1['월'].unique()

(array(['2022'], dtype=object), array(['12'], dtype=object))

In [495]:
yr_list, month_list, grd_list, fuel_list = [], [], [], []
for yr in range(2019, int(year) + 1):
    for month in range(1, 13):
        for grd in ['1', '2', '3', '4', '5', 'X']:
            for fuel in grp1['연료'].unique():
                yr_list.append(str(yr))
                month_list.append(f'{month:0>2}')
                grd_list.append(grd)
                fuel_list.append(fuel)
base = pd.DataFrame({'연도':yr_list, '월':month_list, '배출가스등급':grd_list, '연료':fuel_list})
base

,연도,월,배출가스등급,연료
0,2019,01,1,LPG(액화석유가스)
1,2019,01,1,휘발유
2,2019,01,1,경유
3,2019,01,2,LPG(액화석유가스)
4,2019,01,2,휘발유
...,...,...,...,...
859,2022,12,5,휘발유
860,2022,12,5,경유
861,2022,12,X,LPG(액화석유가스)
862,2022,12,X,휘발유


In [508]:
base['연도'].unique(), base['월'].unique()

(array(['2019', '2020', '2021', '2022'], dtype=object),
 array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
        '12'], dtype=object))

In [496]:
grp2 = dfm2dgl.groupby(['최초등록일자_년', '최초등록일자_월', '배출가스등급', '연료'])['차대번호'].count().reset_index()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '최초등록일자_월':'월', '차대번호':'등록대수'})
grp2

,연도,월,배출가스등급,연료,등록대수
0,1955,06,5,경유,1
1,1958,08,5,휘발유,1
2,1962,03,5,경유,1
3,1962,06,5,경유,5
4,1962,06,5,휘발유,1
...,...,...,...,...,...
3932,2023,06,2,휘발유,4486
3933,2023,06,3,경유,1342
3934,2023,06,X,LPG(액화석유가스),429
3935,2023,06,X,경유,3790


In [510]:
grp2['연도'].unique(), grp2['월'].unique()

(array(['1955', '1958', '1962', '1963', '1964', '1965', '1966', '1967',
        '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
        '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983',
        '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
        '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
        '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
        '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
        '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'],
       dtype=object),
 array(['06', '08', '03', '10', '05', '09', '12', '04', '02', '01', '07',
        '11'], dtype=object))

In [497]:
grp3 = errc2dgl.groupby(['변경일자_년', '변경일자_월', '배출가스등급', '연료'])['차대번호'].count().reset_index()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '변경일자_월':'월', '차대번호':'말소대수'})
grp3

,연도,월,배출가스등급,연료,말소대수
0,2019,01,3,휘발유,2
1,2019,01,X,경유,5
2,2019,02,X,LPG(액화석유가스),5
3,2019,02,X,경유,7
4,2019,02,X,휘발유,1
...,...,...,...,...,...
579,2022,06,5,경유,18590
580,2022,06,5,휘발유,35
581,2022,06,X,LPG(액화석유가스),2
582,2022,06,X,경유,53


In [515]:
grp3['연도'].unique(), grp3['월'].unique()

(array(['2019', '2020', '2021', '2022'], dtype=object),
 array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
        '12'], dtype=object))

In [498]:
base1 = base.merge(grp1, on=['연도', '월', '배출가스등급', '연료'], how='left')
base1

,연도,월,배출가스등급,연료,차량대수
0,2019,01,1,LPG(액화석유가스),NaN
1,2019,01,1,휘발유,NaN
2,2019,01,1,경유,NaN
3,2019,01,2,LPG(액화석유가스),NaN
4,2019,01,2,휘발유,NaN
...,...,...,...,...,...
859,2022,12,5,휘발유,30011.00
860,2022,12,5,경유,1081957.00
861,2022,12,X,LPG(액화석유가스),16686.00
862,2022,12,X,휘발유,172901.00


In [514]:
base1.loc[base1['월'] == '12', ['연도', '배출가스등급', '연료', '차량대수']]

,연도,배출가스등급,연료,차량대수
198,2019,1,LPG(액화석유가스),NaN
199,2019,1,휘발유,NaN
200,2019,1,경유,NaN
201,2019,2,LPG(액화석유가스),NaN
202,2019,2,휘발유,NaN
...,...,...,...,...
859,2022,5,휘발유,30011.00
860,2022,5,경유,1081957.00
861,2022,X,LPG(액화석유가스),16686.00
862,2022,X,휘발유,172901.00


In [499]:
base2 = base1.merge(grp2, on=['연도', '월', '배출가스등급', '연료'], how='left')
base2

,연도,월,배출가스등급,연료,차량대수,등록대수
0,2019,01,1,LPG(액화석유가스),NaN,4513.00
1,2019,01,1,휘발유,NaN,9659.00
2,2019,01,1,경유,NaN,NaN
3,2019,01,2,LPG(액화석유가스),NaN,4342.00
4,2019,01,2,휘발유,NaN,66465.00
...,...,...,...,...,...,...
859,2022,12,5,휘발유,30011.00,NaN
860,2022,12,5,경유,1081957.00,NaN
861,2022,12,X,LPG(액화석유가스),16686.00,1459.00
862,2022,12,X,휘발유,172901.00,16028.00


In [500]:
base3 = base2.merge(grp3, on=['연도', '월', '배출가스등급', '연료'], how='left')
base3

,연도,월,배출가스등급,연료,차량대수,등록대수,말소대수
0,2019,01,1,LPG(액화석유가스),NaN,4513.00,NaN
1,2019,01,1,휘발유,NaN,9659.00,NaN
2,2019,01,1,경유,NaN,NaN,NaN
3,2019,01,2,LPG(액화석유가스),NaN,4342.00,NaN
4,2019,01,2,휘발유,NaN,66465.00,NaN
...,...,...,...,...,...,...,...
859,2022,12,5,휘발유,30011.00,NaN,NaN
860,2022,12,5,경유,1081957.00,NaN,NaN
861,2022,12,X,LPG(액화석유가스),16686.00,1459.00,NaN
862,2022,12,X,휘발유,172901.00,16028.00,NaN


In [520]:
base4 = base3.copy()
base4[['연도', '월']] = base4[['연도', '월']].astype('int')
base4 = base4.sort_values(['배출가스등급', '연료', '연도', '월']).reset_index(drop=True)
base4

,연도,월,배출가스등급,연료,차량대수,등록대수,말소대수
0,2019,1,1,LPG(액화석유가스),NaN,4513.00,NaN
1,2019,2,1,LPG(액화석유가스),NaN,3692.00,NaN
2,2019,3,1,LPG(액화석유가스),NaN,5143.00,NaN
3,2019,4,1,LPG(액화석유가스),NaN,5840.00,NaN
4,2019,5,1,LPG(액화석유가스),NaN,7287.00,NaN
...,...,...,...,...,...,...,...
859,2022,8,X,휘발유,NaN,646.00,NaN
860,2022,9,X,휘발유,NaN,17054.00,NaN
861,2022,10,X,휘발유,NaN,3311.00,NaN
862,2022,11,X,휘발유,NaN,5296.00,NaN


In [521]:
base4[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    848
등록대수    406
말소대수    280
dtype: int64

In [522]:
base4[['차량대수', '등록대수', '말소대수']] = base4[['차량대수', '등록대수', '말소대수']].fillna(0)

In [523]:
n = len(base4['연도'].unique()) * len(base4['월'].unique())
for i in range(base4.shape[0] // n):
    for j in range(2, n+1):
        base4.loc[(i+1)*n - j, '차량대수'] = base4.loc[(i+1)*n - (j-1), '차량대수'] + base4.loc[(i+1)*n - (j-1), '말소대수'] - base4.loc[(i+1)*n - (j-1), '등록대수']

In [524]:
base4.head(20)

,연도,월,배출가스등급,연료,차량대수,등록대수,말소대수
0,2019,1,1,LPG(액화석유가스),359113.00,4513.00,0.00
1,2019,2,1,LPG(액화석유가스),362805.00,3692.00,0.00
2,2019,3,1,LPG(액화석유가스),367948.00,5143.00,0.00
3,2019,4,1,LPG(액화석유가스),373788.00,5840.00,0.00
4,2019,5,1,LPG(액화석유가스),381075.00,7287.00,0.00
5,2019,6,1,LPG(액화석유가스),386703.00,5864.00,236.00
6,2019,7,1,LPG(액화석유가스),391591.00,5652.00,764.00
7,2019,8,1,LPG(액화석유가스),395059.00,4201.00,733.00
8,2019,9,1,LPG(액화석유가스),399021.00,4787.00,825.00
9,2019,10,1,LPG(액화석유가스),403804.00,5819.00,1036.00


In [ ]:
stat1 = base4.copy()
stat1['연월'] = stat1['연도'] + 

In [456]:
die = stat1.loc[stat1['연료'] == '경유'].reset_index(drop=True)
gas = stat1.loc[stat1['연료'] == '휘발유'].reset_index(drop=True)
lpg = stat1.loc[stat1['연료'] == 'LPG(액화석유가스)'].reset_index(drop=True)

In [457]:
die['연분기'] = die['연분기'].astype('int')
gas['연분기'] = gas['연분기'].astype('int')
lpg['연분기'] = lpg['연분기'].astype('int')

In [458]:
# 선형 예측
fit1 = np.polyfit(die['연분기'], die['차량대수'], 1)
fit2 = np.polyfit(gas['연분기'], gas['차량대수'], 1)
fit3 = np.polyfit(lpg['연분기'], lpg['차량대수'], 1)
a1, b1 = fit1
a2, b2 = fit2
a3, b3 = fit3

In [525]:
month_list = []
for yr in range(int(year) + 1, 2036):
    for month in range(1, 13):
        month_list.append(int(str(yr) + f'{month:0>2}'))

In [526]:
month_list[:5]

[202301, 202302, 202303, 202304, 202305]

In [465]:
# BSpline 예측
spl1 = intp.BSpline(die['연분기'], die['차량대수'], 1, extrapolate=True)
spl2 = intp.BSpline(gas['연분기'], gas['차량대수'], 1, extrapolate=True)
spl3 = intp.BSpline(lpg['연분기'], lpg['차량대수'], 1, extrapolate=True)
spl1pred = spl1(qt_list)
spl2pred = spl2(qt_list)
spl3pred = spl3(qt_list)

In [466]:
# akima 예측
aki1 = intp.Akima1DInterpolator(die['연분기'], die['차량대수'])
aki2 = intp.Akima1DInterpolator(gas['연분기'], gas['차량대수'])
aki3 = intp.Akima1DInterpolator(lpg['연분기'], lpg['차량대수'])
aki1pred = aki1(qt_list, extrapolate=True)
aki2pred = aki2(qt_list, extrapolate=True)
aki3pred = aki3(qt_list, extrapolate=True)

ValueError: `x` must be strictly increasing sequence.

In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '경유'
for yr in range(year + 1, 2036):
    pred = a1 * yr + b1
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
die_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '경유_예측':pred_list})
die_pred['경유_예측_BSpline'] = spl1pred
die_pred['경유_예측_Akima'] = aki1pred
die_pred

,연도,연료,경유_예측,경유_예측_BSpline,경유_예측_Akima
0,2023,경유,9891328.00,10061712.00,10156376.11
1,2024,경유,9865352.90,10086965.00,10488168.68
2,2025,경유,9839377.80,10112218.00,10890039.05
3,2026,경유,9813402.70,10137471.00,11315943.56
4,2027,경유,9787427.60,10162724.00,11719838.54
5,2028,경유,9761452.50,10187977.00,12055680.35
6,2029,경유,9735477.40,10213230.00,12277425.32
7,2030,경유,9709502.30,10238483.00,12339029.80
8,2031,경유,9683527.20,10263736.00,12194450.12
9,2032,경유,9657552.10,10288989.00,11797642.64


In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = '휘발유'
for yr in range(year + 1, 2036):
    pred = a2 * yr + b2
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
gas_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '휘발유_예측':pred_list})
gas_pred['휘발유_예측_BSpline'] = spl2pred
gas_pred['휘발유_예측_Akima'] = aki2pred
gas_pred

,연도,연료,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima
0,2023,휘발유,12759346.50,12472513.00,12881770.74
1,2024,휘발유,13160189.70,12915417.00,13505977.46
2,2025,휘발유,13561032.90,13358321.00,14255067.39
3,2026,휘발유,13961876.10,13801225.00,15147441.75
4,2027,휘발유,14362719.30,14244129.00,16201501.77
5,2028,휘발유,14763562.50,14687033.00,17435648.70
6,2029,휘발유,15164405.70,15129937.00,18868283.75
7,2030,휘발유,15565248.90,15572841.00,20517808.17
8,2031,휘발유,15966092.10,16015745.00,22402623.17
9,2032,휘발유,16366935.30,16458649.00,24541130.00


In [ ]:
yr_list = []
fuel_list = []
pred_list = []
fuel = 'LPG'
for yr in range(year + 1, 2036):
    pred = a3 * yr + b3
    yr_list.append(yr)
    fuel_list.append(fuel)
    pred_list.append(pred)
lpg_pred = pd.DataFrame({'연도':yr_list, '연료':fuel_list, 'LPG_예측':pred_list})
lpg_pred['LPG_예측_BSpline'] = spl3pred
lpg_pred['LPG_예측_Akima'] = aki3pred
lpg_pred

,연도,연료,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima
0,2023,LPG,1898243.00,1902560.00,1952460.75
1,2024,LPG,1885073.70,1884782.00,1929016.48
2,2025,LPG,1871904.40,1867004.00,1779271.43
3,2026,LPG,1858735.10,1849226.00,1431104.85
4,2027,LPG,1845565.80,1831448.00,812395.96
5,2028,LPG,1832396.50,1813670.00,-148975.99
6,2029,LPG,1819227.20,1795892.00,-1525131.76
7,2030,LPG,1806057.90,1778114.00,-3388192.11
8,2031,LPG,1792888.60,1760336.00,-5810277.81
9,2032,LPG,1779719.30,1742558.00,-8863509.62


In [ ]:
die_t = pd.concat([die, die_pred], ignore_index=True)
gas_t = pd.concat([gas, gas_pred], ignore_index=True)
lpg_t = pd.concat([lpg, lpg_pred], ignore_index=True)

In [ ]:
die_t = die_t.rename(columns={'차량대수':'경유_대수'})
gas_t = gas_t.rename(columns={'차량대수':'휘발유_대수'})
lpg_t = lpg_t.rename(columns={'차량대수':'LPG_대수'})

In [ ]:
die_t.columns

Index(['연도', '연료', '경유_대수', '경유_예측', '경유_예측_BSpline', '경유_예측_Akima'], dtype='object')

In [ ]:
df5 = pd.concat([die_t[['연도', '경유_대수', '경유_예측', '경유_예측_BSpline', '경유_예측_Akima']], gas_t[['휘발유_대수', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima']], lpg_t[['LPG_대수', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']]], axis=1)
df5.head()

,연도,경유_대수,경유_예측,경유_예측_BSpline,경유_예측_Akima,휘발유_대수,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima,LPG_대수,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima
0,2019,9985953.00,NaN,NaN,NaN,11143801.00,NaN,NaN,NaN,1955894.00,NaN,NaN,NaN
1,2020,10011206.00,NaN,NaN,NaN,11586705.00,NaN,NaN,NaN,1938116.00,NaN,NaN,NaN
2,2021,9887199.00,NaN,NaN,NaN,11934402.00,NaN,NaN,NaN,1908930.00,NaN,NaN,NaN
3,2022,9940705.00,NaN,NaN,NaN,12364046.00,NaN,NaN,NaN,1921725.00,NaN,NaN,NaN
4,2023,NaN,9891328.00,10061712.00,10156376.11,NaN,12759346.50,12472513.00,12881770.74,NaN,1898243.00,1902560.00,1952460.75


In [ ]:
# 음수 0으로 처리
df5.loc[df5['경유_예측'] < 0, '경유_예측'] = 0
df5.loc[df5['경유_예측_BSpline'] < 0, '경유_예측_BSpline'] = 0
df5.loc[df5['경유_예측_Akima'] < 0, '경유_예측_Akima'] = 0
df5.loc[df5['휘발유_예측'] < 0, '휘발유_예측'] = 0
df5.loc[df5['휘발유_예측_BSpline'] < 0, '휘발유_예측_BSpline'] = 0
df5.loc[df5['휘발유_예측_Akima'] < 0, '휘발유_예측_Akima'] = 0
df5.loc[df5['LPG_예측'] < 0, 'LPG_예측'] = 0
df5.loc[df5['LPG_예측_BSpline'] < 0, 'LPG_예측_BSpline'] = 0
df5.loc[df5['LPG_예측_Akima'] < 0, 'LPG_예측_Akima'] = 0

In [ ]:
# 첫째자리에서 반올림
df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']] = df5[['경유_대수', '휘발유_대수', 'LPG_대수', '경유_예측', '경유_예측_BSpline','경유_예측_Akima', '휘발유_예측', '휘발유_예측_BSpline', '휘발유_예측_Akima', 'LPG_예측', 'LPG_예측_BSpline', 'LPG_예측_Akima']].round(0)

In [ ]:
df5

,연도,경유_대수,경유_예측,경유_예측_BSpline,경유_예측_Akima,휘발유_대수,휘발유_예측,휘발유_예측_BSpline,휘발유_예측_Akima,LPG_대수,LPG_예측,LPG_예측_BSpline,LPG_예측_Akima
0,2019,9985953.00,NaN,NaN,NaN,11143801.00,NaN,NaN,NaN,1955894.00,NaN,NaN,NaN
1,2020,10011206.00,NaN,NaN,NaN,11586705.00,NaN,NaN,NaN,1938116.00,NaN,NaN,NaN
2,2021,9887199.00,NaN,NaN,NaN,11934402.00,NaN,NaN,NaN,1908930.00,NaN,NaN,NaN
3,2022,9940705.00,NaN,NaN,NaN,12364046.00,NaN,NaN,NaN,1921725.00,NaN,NaN,NaN
4,2023,NaN,9891328.00,10061712.00,10156376.00,NaN,12759347.00,12472513.00,12881771.00,NaN,1898243.00,1902560.00,1952461.00
5,2024,NaN,9865353.00,10086965.00,10488169.00,NaN,13160190.00,12915417.00,13505977.00,NaN,1885074.00,1884782.00,1929016.00
6,2025,NaN,9839378.00,10112218.00,10890039.00,NaN,13561033.00,13358321.00,14255067.00,NaN,1871904.00,1867004.00,1779271.00
7,2026,NaN,9813403.00,10137471.00,11315944.00,NaN,13961876.00,13801225.00,15147442.00,NaN,1858735.00,1849226.00,1431105.00
8,2027,NaN,9787428.00,10162724.00,11719839.00,NaN,14362719.00,14244129.00,16201502.00,NaN,1845566.00,1831448.00,812396.00
9,2028,NaN,9761453.00,10187977.00,12055680.00,NaN,14763562.00,14687033.00,17435649.00,NaN,1832397.00,1813670.00,0.00


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
df5['테이블생성일자'] = today_date

In [ ]:
die = 

# code end